# Cargar librerias

In [1]:
import pandas as pd
import openpyxl
import re
import os
from openpyxl import load_workbook, Workbook
import numpy as np
from numpy import ceil 
from datetime import date
import math
from datetime import datetime
import openpyxl
from openpyxl import load_workbook

##   FUNCIONES - CAMBIAR MES  

In [2]:
def calcular_saldo_acumulado(data, mes_actual):
    # Filtrar las filas correspondientes a los meses anteriores y el mes actual
    columnas = [f'Saldo Periodo {str(i).zfill(2)}' for i in range(1, 17)]
    data_filtrada = data[columnas[:mes_actual]]

    # Sumar los saldos de los meses seleccionados
    saldo_acumulado = data_filtrada.sum(axis=1)
    
    # Devolver el saldo acumulado
    return saldo_acumulado

## ESTABLECER LA RUTA DE LOS ARCHIVOS 

In [219]:
# Establecer la ruta de trabajo
ruta1 = 'D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/GENERALES/Bases/'

os.chdir(ruta1)

# Configurar opciones de estilo predeterminadas
pd.options.display.float_format = '{:,.2f}'.format

# Configurar opciones de formato de fecha y hora
date_format = "%m/%d/%Y"
datetime_format = "%Y-%m-%d %H:%M:%S"

# Cambiar el color de borde predeterminado
border_colour = "#4F81BD"

#  CÓDIGO PARA CARGAR Y LEER LAS BD EN FORMATO.XLSM

In [220]:
# Ruta del archivo principal
ruta_excel = "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/Fuentes de Información COGN.xlsx"

# Lectura básica de hojas comunes
VALIDACION_RAMOS = pd.read_excel(ruta_excel, sheet_name="VALIDACION RAMOS").drop_duplicates()
AÑO_ACTUAL = pd.read_excel(ruta_excel, sheet_name="AÑO ACTUAL")
N1 = pd.read_excel(ruta_excel, sheet_name="N-1")
N2 = pd.read_excel(ruta_excel, sheet_name="N-2")
N3 = pd.read_excel(ruta_excel, sheet_name="N-3")
BALANCE = pd.read_excel(ruta_excel, sheet_name="BALANCE")
CUIF = pd.read_excel(ruta_excel, sheet_name="CUIF")
homologacion = pd.read_excel(ruta_excel, sheet_name="HOMOLOGACION")
fechas = pd.read_excel(ruta_excel, sheet_name="FECHAS")
Ramos = pd.read_excel(ruta_excel, sheet_name="Ramos")

# Lectura con ajuste de columnas y filas para 'ACTIVOS'
activos = pd.read_excel(ruta_excel, sheet_name="ACTIVOS", header=None)
headers = ['Indice', 'Nombre o Razón Social', 'Código PUC', 'Clase de Inversión', 
           'Valor Mercado ($)', 'Calificación Riesgo Crediticio', 'Ramo', 'SUBCUENTA', 'CATEGORIA']
activos.columns = headers
activos = activos.iloc[4:].reset_index(drop=True)
activos = activos.drop('Indice', axis=1)
activos['Valor Mercado ($)'] = pd.to_numeric(activos['Valor Mercado ($)'])

# Lectura de reaseguros
bd_reaseguros = pd.read_excel(ruta_excel, sheet_name="REASEGURO", skiprows=11, usecols="B:N", nrows=688)

# Lectura secciones específicas de COASEGURO
bd_COASEGURADORES_CUENTA_CORRIENTE_ACEPTADOS = pd.read_excel(
    ruta_excel, sheet_name="COASEGURO", skiprows=8, usecols="B:G", nrows=31)

bd_COASEGURADORES_CUENTA_CORRIENTE_CEDIDOS = pd.read_excel(
    ruta_excel, sheet_name="COASEGURO", skiprows=41, usecols="B:G", nrows=31)

bd_DETERIORO_CUENTAS_POR_COBRAR_ACTIVIDAD_ASEGURADORA_COASEGURADORES = pd.read_excel(
    ruta_excel, sheet_name="COASEGURO", skiprows=74, usecols="B:G", nrows=15)

# Lectura de la hoja INFO y extracción de valores clave
Info = pd.read_excel(ruta_excel, sheet_name="INFO")
UVR = float(Info.iloc[6, 2])
mes_s = Info.iloc[4, 2]
meses = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 
         'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
mes = meses.index(mes_s) + 1 if mes_s in meses else None
Info['irow'] = range(1, len(Info) + 1)

# Archivo adicional
T_homologacion_ramos = pd.read_excel("T_homologacion_ramos.xlsx")


#   Data.frame de análisis de cuentas

In [5]:
# Crear listas vacías
bd_analisis_cuentas = []
estructura = []

In [6]:
# Cambiar el directorio de trabajo
os.chdir('D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/GENERALES/Bases/Proforma')

# Cargar el archivo Excel existente
wb = load_workbook("Plantilla_F.3000-76.xlsx")

# Crear un nuevo libro de trabajo
new_wb = Workbook()

#  DIFERENCIADORES 

In [7]:
# Extraer y agregar primeros 4 y 6 dígitos para N2
N2['4d'] = N2['Cuenta alternativa'].astype(str).str[:4].astype(float)
N2['6d'] = N2['Cuenta alternativa'].astype(str).str[:6].astype(float)

# Extraer y agregar primeros 4 y 6 dígitos para N3
N3['4d'] = N3['Cuenta alternativa'].astype(str).str[:4].astype(float)
N3['6d'] = N3['Cuenta alternativa'].astype(str).str[:6].astype(float)

# Extraer y agregar primeros 4 y 6 dígitos para AÑO_ACTUAL
AÑO_ACTUAL['4d'] = AÑO_ACTUAL['Cuenta alternativa'].astype(str).str[:4].astype(float)
AÑO_ACTUAL['6d'] = AÑO_ACTUAL['Cuenta alternativa'].astype(str).str[:6].astype(float)

# Calcular acumulado para N3
N3['acumulado_mes'] = calcular_saldo_acumulado(N3, mes)

# Calcular saldos mes pasado para AÑO_ACTUAL
AÑO_ACTUAL['Saldo mes pasado'] = AÑO_ACTUAL['Total Cuenta'] - AÑO_ACTUAL.iloc[:, mes+14]

# Calcular saldos mes pasado para N1
N1['Saldo mes pasado'] = N1['Total Cuenta'] - N1.iloc[:, mes+14]

# Calcular saldos mes pasado para N2
N2['Saldo mes pasado'] = N2['Total Cuenta'] - N2.iloc[:, mes+14]

# Calcular saldos mes pasado para N3
N3['Saldo mes pasado'] = N3['Total Cuenta'] - N3.iloc[:, mes+14]

# 1005 

In [8]:
# Inicializar bd_analisis_cuentas como DataFrame vacío
bd_analisis_cuentas = pd.DataFrame()

# Convertir columnas a tipo string
AÑO_ACTUAL["Cuenta alternativa"] = AÑO_ACTUAL["Cuenta alternativa"].astype(str)
AÑO_ACTUAL["4d"] = AÑO_ACTUAL["4d"].astype(str)

# Filtrar filas que contienen "3105"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('3105')]

# Crear a1005 usando operaciones de pandas
a1005 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 1, 
                'Subcuenta': '005'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Agregar columna notas
a1005['notas'] = ""

# Agregar a1005 a bd_analisis_cuentas
bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a1005], ignore_index=True)

# Calcular s1005
s1005 = abs(a1005['Saldos'].sum())

#  1040

In [9]:
# Filtrar filas que contienen "3805"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('3805')]

# Crear a1040 usando operaciones de pandas
a1040 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 1, 
                'Subcuenta': '040'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a1040 antes de proceder
if len(a1040) > 0:
    # Agregar columna notas
    a1040['notas'] = ""
    
    # Agregar a1040 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a1040], ignore_index=True)

# Calcular s1040
s1040 = abs(a1040['Saldos'].sum())

# 1055

In [10]:
# Filtrar filas que contienen "3205"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('3205')]

# Crear a1055 usando operaciones de pandas
a1055 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 1, 
                'Subcuenta': '055'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a1055 antes de proceder
if len(a1055) > 0:
    # Agregar columna notas
    a1055['notas'] = ""
    
    # Agregar a1055 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a1055], ignore_index=True)

# Calcular s1055
s1055 = abs(a1055['Saldos'].sum())

# 1060

In [11]:
# Filtrar filas que contienen "3215"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('3215')]

# Crear a1060 usando operaciones de pandas
a1060 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 1, 
                'Subcuenta': '060'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a1060 antes de proceder
if len(a1060) > 0:
    # Agregar columna notas
    a1060['notas'] = ""
    
    # Agregar a1060 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a1060], ignore_index=True)

# Calcular s1060
s1060 = abs(a1060['Saldos'].sum())

# 1070

In [12]:
# Filtrar filas que contienen "3120"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('3120')]

# Crear a1070 usando operaciones de pandas
a1070 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 1, 
                'Subcuenta': '070'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a1070 antes de proceder
if len(a1070) > 0:
    # Agregar columna notas
    a1070['notas'] = ""
    
    # Agregar a1070 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a1070], ignore_index=True)

# Calcular s1070
s1070 = abs(a1070['Saldos'].sum())

# V1075

In [13]:
# Filtrar filas que contienen "3730"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('3730')]

# Crear a1075 usando operaciones de pandas
a1075 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 1, 
                'Subcuenta': '075'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a1075 antes de proceder
if len(a1075) > 0:
    # Agregar columna notas
    a1075['notas'] = ""
    
    # Agregar a1075 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a1075], ignore_index=True)

# Calcular s1075
s1075 = abs(a1075['Saldos'].sum())

# V1085

In [14]:
# Filtrar filas que contienen "3705"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('3705')]

# Crear a1085 usando operaciones de pandas
a1085 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 1, 
                'Subcuenta': '085'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a1085 antes de proceder
if len(a1085) > 0:
    # Agregar columna notas
    a1085['notas'] = ""
    
    # Agregar a1085 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a1085], ignore_index=True)

# Calcular s1085
s1085 = abs(a1085['Saldos'].sum())

# V1090

In [15]:
# Filtrar filas que contienen "3815"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('3815')]

# Crear a1090 usando operaciones de pandas
a1090 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 1, 
                'Subcuenta': '090'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a1090 antes de proceder
if len(a1090) > 0:
    # Agregar columna notas
    a1090['notas'] = ""
    
    # Agregar a1090 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a1090], ignore_index=True)

# Calcular s1090
s1090 = abs(a1090['Saldos'].sum())

# V1095

In [16]:
# Filtrar filas que contienen "865" en la columna Pos_balance
mask = BALANCE['Pos_balance'].notna() & BALANCE['Pos_balance'].astype(str).str.contains('865')
BalanceSubset = BALANCE[mask]

# Obtener el primer valor de Total_periodos_de_informe y multiplicarlo por -1
s1095 = -1 * BalanceSubset['Total_periodos_de_informe'].iloc[0]

# V1100

In [18]:
# Filtrar filas que contienen "3905"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('3905')]

# Crear a1100 usando operaciones de pandas
a1100 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 1, 
                'Subcuenta': '0100'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a1100 antes de proceder
if len(a1100) > 0:
    # Agregar columna notas
    a1100['notas'] = ""
    
    # Agregar a1100 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a1100], ignore_index=True)

# Calcular s1100
s1100 = abs(a1100['Saldos'].sum())

# DATAFRAME    

In [19]:
# Crear vector con los valores
v1998 = [s1005, s1040, s1055, s1060, s1070, s1075, s1085, s1090, s1095, s1100]

# Crear DataFrame
unidad1998 = pd.DataFrame(v1998, columns=['Saldos'])

# Redondear valores y tomar valor absoluto
unidad1998['Saldos'] = unidad1998['Saldos'].round(0).abs()

# Agregar la suma al final del DataFrame
unidad1998.loc[len(unidad1998)] = unidad1998['Saldos'].sum()

# Agregar columna de Subcuenta
unidad1998['Subcuenta'] = ['005', '040', '055', '060', '070', '075', '085', '090', '095', '100', '998']

# Crear subcuentas (excluyendo s1095 que no está en la lista original)
subcuentas = pd.DataFrame({
    'Saldos': [s1005, s1040, s1055, s1060, s1070, s1085, s1090, s1100]
})

# Hacer merge
t = pd.merge(subcuentas, unidad1998)

# 2005

In [20]:
# Filtrar filas que contienen "3910"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('3910', na=False)]

# Crear a2005 usando operaciones de pandas
a2005 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 2, 
                'Subcuenta': '005'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a2005 antes de proceder
if len(a2005) > 0:
    # Agregar columna notas
    a2005['notas'] = ""
    
    # Agregar a2005 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a2005], ignore_index=True)

# Calcular s2005 usando Decimal para mayor precisión
from decimal import Decimal
s2005 = abs(sum(Decimal(str(x)) for x in a2005['Saldos']))

# 2010

In [21]:
# Filtrar filas que contienen "3920"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('3920', na=False)]

# Crear a2010 usando operaciones de pandas
a2010 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 2, 
                'Subcuenta': '010'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a2010 antes de proceder
if len(a2010) > 0:
    # Agregar columna notas
    a2010['notas'] = ""
    
    # Agregar a2010 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a2010], ignore_index=True)

# Calcular s2010 usando Decimal para mayor precisión
from decimal import Decimal
s2010 = abs(sum(Decimal(str(x)) for x in a2010['Saldos']))

# 2015

In [22]:
# Filtrar filas que contienen "3930"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('3930', na=False)]

# Crear a2015 usando operaciones de pandas
a2015 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 2, 
                'Subcuenta': '015'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a2015 antes de proceder
if len(a2015) > 0:
    # Agregar columna notas
    a2015['notas'] = ""
    
    # Agregar a2015 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a2015], ignore_index=True)

# Calcular s2015 usando Decimal para mayor precisión
from decimal import Decimal
s2015 = abs(sum(Decimal(str(x)) for x in a2015['Saldos']))

# 2025

In [23]:
# Buscar el valor correspondiente a "Inversión en compañías de seguros de vida"
s2025 = None
mascara = Info['Unnamed: 1'] == 'Inversión en compañías de seguros de vida'
if any(mascara):
    valor = Info.loc[mascara, 'Unnamed: 2'].iloc[0]
    if not pd.isna(valor):
        s2025 = round(float(str(valor).replace(',', '')))

# 2040

In [24]:
# Filtrar filas que contienen "1910" o "2558"
cons1 = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('1910', na=False)]
cons2 = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('2558', na=False)]

# Combinar los resultados
AÑO_ACTUALSubset = pd.concat([cons1, cons2])

# Crear a2040 usando operaciones de pandas
a2040 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 2, 
                'Subcuenta': '040'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a2040 antes de proceder
if len(a2040) > 0:
    # Agregar columna notas
    a2040['notas'] = ""
    
    # Agregar a2040 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a2040], ignore_index=True)

# Calcular s2040 usando Decimal para mayor precisión
from decimal import Decimal
s2040 = sum(Decimal(str(x)) for x in a2040['Saldos'])  # Note que aquí no usamos abs()

# 2045

In [25]:
# Extraer el valor cuando irow es 25 (equivalente a ifelse en R)
s2045 = Info.loc[Info['irow'] == 25, 'Unnamed: 2'].values if any(Info['irow'] == 25) else None

# Eliminar valores NA y convertir a numérico (equivalente a na.omit y as.numeric en R)
if s2045 is not None and len(s2045) > 0:
    # Convertir a string, reemplazar comas, convertir a float y redondear
    try:
        s2045 = round(float(str(s2045[0]).replace(',', '')))
    except (ValueError, TypeError):
        s2045 = None
else:
    s2045 = None

# 2055

In [26]:
# Filtrar filas que contienen "1911"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('1911', na=False)]

# Crear a2055 usando operaciones de pandas
a2055 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 2, 
                'Subcuenta': '055'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a2055 antes de proceder
if len(a2055) > 0:
    # Agregar columna notas
    a2055['notas'] = ""
    
    # Agregar a2055 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a2055], ignore_index=True)

# Calcular s2055 usando Decimal para mayor precisión
from decimal import Decimal
suma_saldos = sum(Decimal(str(x)) for x in a2055['Saldos'])
s2055 = suma_saldos - (s2045 if s2045 is not None else Decimal('0'))

# 2075

In [27]:
# Asumiendo que tienes los DataFrames AÑO_ACTUAL e Info
s2075 = 9153832127
v2075 = 9153832127

# Si necesitas mantener el formato de DataFrame
v2075 = pd.DataFrame({
    'Unidad de captura': [2],
    'Subcuenta': ['075'],
    'saldos': [9153832127]
})

# DATAFRAME

In [28]:
from decimal import Decimal

# Convertir todos los valores a Decimal
s2998 = [Decimal(str(x)) if x is not None else Decimal('0') for x in [s2005, s2010, s2015, s2025, s2040, s2045, s2055, s2075]]

# Crear DataFrame
unidad2998 = pd.DataFrame({'Saldos': s2998})

# Redondear valores y convertir a entero
unidad2998['Saldos'] = unidad2998['Saldos'].apply(lambda x: round(x))

# Aplicar valor absoluto
unidad2998['Saldos'] = unidad2998['Saldos'].apply(lambda x: abs(x))

# Calcular la suma específica y agregarla como nueva fila
indices_suma = [0,1,2,4,5,6,7]
suma_especifica = sum(unidad2998['Saldos'].iloc[i] for i in indices_suma)
unidad2998.loc[8] = suma_especifica

# Agregar columna de Subcuenta
unidad2998['Subcuenta'] = ['005', '010', '015', '025', '040', '045', '055', '075', '998']

# Reordenar columnas
unidad2998 = unidad2998[['Saldos', 'Subcuenta']]

# Crear subcuentas
subcuentas = pd.Series([s2005, s2010, s2015, s2040])

# LIMPIEZA DE LOS DATOS

In [29]:
# Eliminar todas las variables mencionadas si existen
for var in ['s1005', 's1040', 's1055', 's1060', 's1070', 's1075', 's1085', 's1090', 
            's1100', 's2005', 's2010', 's2015', 's2040', 's2040_1', 's2040_2', 's2075', 
            'v1998', 'v2005ca', 'v2010ca', 'v2015ca', 'v2025', 'v2040ca', 'v2045', 
            'V2055', 'v2075', 'v2998', 'v2055', 'v2075', 't2']:
    if var in globals():
        del globals()[var]

#  PARTE 4 

In [30]:
# Obtener el valor de la quinta fila (índice 4) de la columna 'Saldos'
s4030 = unidad2998['Saldos'].iloc[4]

# DATAFRAME

In [31]:
# Crear lista inicial con s4030
v4998 = [s4030]

# Calcular la suma
v4998_1 = sum(v4998)

# Agregar la suma a la lista
v4998 = [s4030, v4998_1]

# Crear DataFrame
unidad4998 = pd.DataFrame({
    'Saldos': v4998,
    'Subcuenta': ['030', '999']
})

# Opcional: asegurar el orden de las columnas
unidad4998 = unidad4998[['Saldos', 'Subcuenta']]

 # SALDOS ACUMULADOS AÑO PASADO

In [32]:
# Asumiendo que calcular_saldo_acumulado es una función que ya existe
acumulado_aa = calcular_saldo_acumulado(N1, mes)
N1['acumulado_mes'] = acumulado_aa

# 5010

In [33]:
# Filtrar filas que contienen "412140"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('412140', na=False)]

# Crear a5010 usando operaciones de pandas
a5010 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 5, 
                'Subcuenta': '010'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a5010 antes de proceder
if len(a5010) > 0:
    # Agregar columna notas
    a5010['notas'] = ""
    
    # Agregar a5010 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a5010], ignore_index=True)

# Calcular s5010 usando valor absoluto de la suma
from decimal import Decimal
s5010 = abs(sum(Decimal(str(x)) for x in a5010['Saldos']))

#  PASADO VENCIDO

In [34]:
# Crear columnas de 4 y 6 dígitos
N1['4d'] = N1['Cuenta alternativa'].astype(str).str[:4].astype(float)
N1['6d'] = N1['Cuenta alternativa'].astype(str).str[:6].astype(float)

# Filtrar filas que contienen "412140"
N1Subset = N1[N1['6d'].astype(str).str.contains('412140', na=False)]

# Crear v5010_1 usando operaciones de pandas
v5010_1 = (N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
           .rename(columns={'Total Cuenta': 'Saldos', 
                           'Saldo mes pasado': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
               **{'Unidad de captura': 5, 
                  'Subcuenta': '010'}
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                           'saldos_ant': 'Saldos del mes pasado'})
           .assign(
               **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                  'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
           )
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
           .reset_index(drop=True))

# Verificar si hay filas en v5010_1 antes de proceder
if len(v5010_1) > 0:
    # Agregar columna notas
    v5010_1['notas'] = ""
    
    # Agregar v5010_1 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5010_1], ignore_index=True)

# Calcular s5010_1 usando valor absoluto de la suma
from decimal import Decimal
s5010_1 = abs(sum(Decimal(str(x)) for x in v5010_1['Saldos']))

# PASADO MES

In [35]:
# Filtrar filas que contienen "412140"
N1Subset = N1[N1['6d'].astype(str).str.contains('412140', na=False)]

# Crear v5010_2 usando operaciones de pandas
v5010_2 = (N1Subset[['Cuenta alternativa', 'acumulado_mes', 'Saldo mes pasado', 'Ejercicio']]
           .rename(columns={'acumulado_mes': 'Saldos', 
                           'Saldo mes pasado': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
               **{'Unidad de captura': 5, 
                  'Subcuenta': '010'}
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                           'saldos_ant': 'Saldos del mes pasado'})
           .assign(
               **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                  'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
           )
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
           .reset_index(drop=True))

# Verificar si hay filas en v5010_2 antes de proceder
if len(v5010_2) > 0:
    # Agregar columna notas
    v5010_2['notas'] = ""
    
    # Agregar v5010_2 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5010_2], ignore_index=True)

# Calcular s5010_2 (nota: sin valor absoluto en este caso)
s5010_2 = sum(v5010_2['Saldos'])

# 5020

In [36]:
# Filtrar filas que contienen "412155"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('412155', na=False)]

# Crear a5020 usando operaciones de pandas
a5020 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
             **{'Unidad de captura': 5, 
                'Subcuenta': '020'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a5020 antes de proceder
if len(a5020) > 0:
    # Agregar columna notas
    a5020['notas'] = ""
    
    # Agregar a5020 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a5020], ignore_index=True)

# Calcular s5020 (sin valor absoluto)
s5020 = sum(a5020['Saldos'])

# PASADO VENCIDO 

In [37]:
# Crear columnas de 4 y 6 dígitos
N1['4d'] = N1['Cuenta alternativa'].astype(str).str[:4].astype(float)
N1['6d'] = N1['Cuenta alternativa'].astype(str).str[:6].astype(float)

# Filtrar filas que contienen "412155"
N1Subset = N1[N1['6d'].astype(str).str.contains('412155', na=False)]

# Crear v5020_1 usando operaciones de pandas
v5020_1 = (N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
           .rename(columns={'Total Cuenta': 'Saldos', 
                           'Saldo mes pasado': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
               **{'Unidad de captura': 5, 
                  'Subcuenta': '020'}
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                           'saldos_ant': 'Saldos del mes pasado'})
           .assign(
               **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                  'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
           )
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
           .reset_index(drop=True))

# Verificar si hay filas en v5020_1 antes de proceder
if len(v5020_1) > 0:
    # Agregar columna notas
    v5020_1['notas'] = ""
    
    # Agregar v5020_1 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5020_1], ignore_index=True)

# Calcular s5020_1 (sin valor absoluto)
s5020_1 = sum(v5020_1['Saldos'])

# PASADO MES

In [38]:
# Filtrar filas que contienen "412155"
N1Subset = N1[N1['6d'].astype(str).str.contains('412155', na=False)]

# Crear v5020_2 usando operaciones de pandas
v5020_2 = (N1Subset[['Cuenta alternativa', 'acumulado_mes', 'Saldo mes pasado', 'Ejercicio']]
           .rename(columns={'acumulado_mes': 'Saldos', 
                           'Saldo mes pasado': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
               **{'Unidad de captura': 5, 
                  'Subcuenta': '020'}
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                           'saldos_ant': 'Saldos del mes pasado'})
           .assign(
               **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                  'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
           )
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
           .reset_index(drop=True))

# Verificar si hay filas en v5020_2 antes de proceder
if len(v5020_2) > 0:
    # Agregar columna notas
    v5020_2['notas'] = ""
    
    # Agregar v5020_2 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5020_2], ignore_index=True)

# Calcular s5020_2 (multiplicar por -1)
s5020_2 = sum(v5020_2['Saldos']) * -1

# 5005

In [39]:
# Filtrar y combinar subconjuntos
codigos = ['412105', '412110', '412145', '412150']
AÑO_ACTUALSubset = pd.concat([
    AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear a5005 usando operaciones de pandas
a5005 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
             **{'Unidad de captura': 5, 
                'Subcuenta': '005'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a5005 antes de proceder
if len(a5005) > 0:
    # Agregar columna notas
    a5005['notas'] = ""
    
    # Agregar a5005 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a5005], ignore_index=True)

# Calcular s5005 usando valor absoluto de la suma
s5005 = abs(sum(a5005['Saldos']))

#  PASADO VENCIDO 

In [40]:
# Filtrar y combinar subconjuntos
codigos = ['412105', '412110', '412145', '412150']
N1Subset = pd.concat([
    N1[N1['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear v5005_1 usando operaciones de pandas
v5005_1 = (N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
           .rename(columns={'Total Cuenta': 'Saldos', 
                           'Saldo mes pasado': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
               **{'Unidad de captura': 5, 
                  'Subcuenta': '005'}
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                           'saldos_ant': 'Saldos del mes pasado'})
           .assign(
               **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                  'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
           )
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
           .reset_index(drop=True))

# Verificar si hay filas en v5005_1 antes de proceder
if len(v5005_1) > 0:
    # Agregar columna notas
    v5005_1['notas'] = ""
    
    # Agregar v5005_1 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5005_1], ignore_index=True)

# Calcular s5005_1 usando valor absoluto de la suma
s5005_1 = abs(sum(v5005_1['Saldos']))

#  PASADO MES

In [41]:
# Extraer las columnas necesarias
v5005_2 = N1Subset['acumulado_mes']
v5005_2ca = N1Subset['Cuenta alternativa']

# Calcular la suma con valor absoluto y multiplicar por -1
s5005_2 = abs(sum(v5005_2)) * -1

# Crear el DataFrame
v5005_2 = pd.DataFrame({
    'Cuenta alternativa': v5005_2ca,
    'saldos': v5005_2
})

# Extraer los primeros 6 dígitos de Cuenta alternativa
v5005_2['Cuenta'] = v5005_2['Cuenta alternativa'].astype(str).str[:6]

# Hacer el matching con CUIF
v5005_2['Nombre Cuenta'] = v5005_2['Cuenta'].map(dict(zip(CUIF['CUENTA'], CUIF['NOMBRE'])))

# 5015

In [42]:
# Filtrar y combinar subconjuntos
codigos = ['414205', '414225']
AÑO_ACTUALSubset = pd.concat([
    AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear a5015 usando operaciones de pandas
a5015 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
             **{'Unidad de captura': 5, 
                'Subcuenta': '015'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a5015 antes de proceder
if len(a5015) > 0:
    # Agregar columna notas
    a5015['notas'] = ""
    
    # Agregar a5015 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a5015], ignore_index=True)

# Calcular s5015 (sin valor absoluto)
s5015 = sum(a5015['Saldos'])

# PASADO VENCIDO

In [43]:
# Filtrar y combinar subconjuntos
codigos = ['414205', '414225']
N1Subset = pd.concat([
    N1[N1['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear v5015 usando operaciones de pandas
v5015 = (N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
             **{'Unidad de captura': 5, 
                'Subcuenta': '015'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en v5015 antes de proceder
if len(v5015) > 0:
    # Agregar columna notas
    v5015['notas'] = ""
    
    # Agregar v5015 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5015], ignore_index=True)

# Calcular s5015_1
s5015_1 = sum(v5015['Saldos'])

# PASADO MES

In [44]:
# Extraer las columnas necesarias
v5015_2 = N1Subset['acumulado_mes']
v5015_2ca = N1Subset['Cuenta alternativa']

# Calcular la suma
s5015_2 = sum(v5015_2)

# Crear el DataFrame
v5015_2 = pd.DataFrame({
    'Cuenta alternativa': v5015_2ca,
    'saldos': v5015_2
})

# Extraer los primeros 6 dígitos de Cuenta alternativa
v5015_2['Cuenta'] = v5015_2['Cuenta alternativa'].astype(str).str[:6]

# Hacer el matching con CUIF
v5015_2['Nombre Cuenta'] = v5015_2['Cuenta'].map(dict(zip(CUIF['CUENTA'], CUIF['NOMBRE'])))

# 5025

In [45]:
# Filtrar y combinar subconjuntos
codigos = ['515180', '515475']
AÑO_ACTUALSubset = pd.concat([
    AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear a5025 usando operaciones de pandas
a5025 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
             **{'Unidad de captura': 5, 
                'Subcuenta': '025'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a5025 antes de proceder
if len(a5025) > 0:
    # Agregar columna notas
    a5025['notas'] = ""
    
    # Agregar a5025 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a5025], ignore_index=True)

# Calcular s5025
s5025 = sum(a5025['Saldos'])

# PASADO VENCIDO

In [46]:
# Filtrar y combinar subconjuntos
codigos = ['515180', '515475']
N1Subset = pd.concat([
    N1[N1['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear v5025_1 usando operaciones de pandas
v5025_1 = (N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
           .rename(columns={'Total Cuenta': 'Saldos', 
                           'Saldo mes pasado': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
               **{'Unidad de captura': 5, 
                  'Subcuenta': '025'}
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                           'saldos_ant': 'Saldos del mes pasado'})
           .assign(
               **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                  'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
           )
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
           .reset_index(drop=True))

# Verificar si hay filas en v5025_1 antes de proceder
if len(v5025_1) > 0:
    # Agregar columna notas
    v5025_1['notas'] = ""
    
    # Agregar v5025_1 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5025_1], ignore_index=True)

# Calcular s5025_1
s5025_1 = sum(v5025_1['Saldos'])

#  PASADO MES

In [47]:
# Extraer las columnas necesarias
v5025_2 = N1Subset['acumulado_mes']
v5025_2ca = N1Subset['Cuenta alternativa']

# Calcular la suma
s5025_2 = sum(v5025_2)

# Crear el DataFrame
v5025_2 = pd.DataFrame({
    'Cuenta alternativa': v5025_2ca,
    'saldos': v5025_2
})

# Extraer los primeros 6 dígitos de Cuenta alternativa
v5025_2['Cuenta'] = v5025_2['Cuenta alternativa'].astype(str).str[:6]

# Hacer el matching con CUIF
v5025_2['Nombre Cuenta'] = v5025_2['Cuenta'].map(dict(zip(CUIF['CUENTA'], CUIF['NOMBRE'])))

# SUMATORIAS 

In [48]:
from decimal import Decimal

# Convertir todos los valores a Decimal antes de sumar
p5005 = Decimal(str(s5005)) + Decimal(str(s5005_1)) + Decimal(str(s5005_2))
p5010 = Decimal(str(s5010)) + Decimal(str(s5010_1)) + Decimal(str(s5010_2))
p5015 = Decimal(str(s5015)) + Decimal(str(s5015_1)) + Decimal(str(s5015_2))
p5020 = Decimal(str(s5020)) + Decimal(str(s5020_1)) + Decimal(str(s5020_2))
p5025 = Decimal(str(s5025)) + Decimal(str(s5025_1)) + Decimal(str(s5025_2))

# DATAFRAME 

In [49]:
# Crear el primer DataFrame
subcuentas05 = ['005', '010', '015', '020', '025']
saldos05 = [float(p5005), float(p5010), float(p5015), float(p5020), float(p5025)]
unidad5998 = pd.DataFrame({
    'subcuentas05': subcuentas05,
    'saldos05': saldos05
})

# Redondear valores y aplicar valor absoluto
unidad5998['saldos05'] = unidad5998['saldos05'].round(0).abs()
unidad5998['saldos05'] = unidad5998['saldos05'].fillna(0)

# Cálculos para los nuevos valores
suma_1_3 = unidad5998['saldos05'].iloc[0:3].sum()
suma_4_5 = unidad5998['saldos05'].iloc[3:5].sum()
diferencia = suma_1_3 - suma_4_5

# Calcular v5030 y v5035
v5030 = min(UVR * 41000000, diferencia) * 0.18
v5035 = round(max((diferencia - UVR * 41000000) * 0.16, 0))
v5998 = round(v5030 + v5035, 0)

# Crear el segundo DataFrame y concatenar
unidad5998_2 = pd.DataFrame({
    'subcuentas05': ['030', '035', '998'],
    'saldos05': [v5030, v5035, v5998]
})

# Combinar los DataFrames
unidad5998 = pd.concat([unidad5998, unidad5998_2], ignore_index=True)

# 6020

In [50]:
# Filtrar y combinar subconjuntos
codigos = ['168005', '168010']
AÑO_ACTUALSubset = pd.concat([
    AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear a6020 usando operaciones de pandas
a6020 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
             **{'Unidad de captura': 6, 
                'Subcuenta': '020'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a6020 antes de proceder
if len(a6020) > 0:
    # Agregar columna notas
    a6020['notas'] = ""
    
    # Agregar a6020 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a6020], ignore_index=True)

# Calcular s6020
s6020 = sum(a6020['Saldos'])

# 6015

In [51]:
# Filtrar y combinar subconjuntos
codigos = ['265005', '265025', '168005', '168010']
AÑO_ACTUALSubset = pd.concat([
    AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear a6015 usando operaciones de pandas
a6015 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
             **{'Unidad de captura': 6, 
                'Subcuenta': '015'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a6015 antes de proceder
if len(a6015) > 0:
    # Agregar columna notas
    a6015['notas'] = ""
    
    # Asegurar que los nombres de columnas coincidan con bd_analisis_cuentas
    a6015.columns = bd_analisis_cuentas.columns
    
    # Agregar a6015 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a6015], ignore_index=True)

# Calcular s6015 (valor absoluto y truncado) con redondeo
s6015 = int(round(abs(sum(a6015['Saldos'])), 0))

# 6025  PASADO VENCIDO

In [52]:
# Filtrar y combinar subconjuntos
codigos = ['265005', '265025', '168005', '168010']
N3Subset = pd.concat([
    N3[N3['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear v6025_2 usando operaciones de pandas
v6025_2 = (N3Subset[['Cuenta alternativa', 'acumulado_mes', 'Arrastre saldos', 'Ejercicio']]
           .rename(columns={'acumulado_mes': 'Saldos', 
                           'Arrastre saldos': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
               **{'Unidad de captura': 6, 
                  'Subcuenta': '025'}
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                           'saldos_ant': 'Saldos del mes pasado'})
           .assign(
               **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                  'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
           )
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
           .reset_index(drop=True))

# Verificar si hay filas en v6025_2 antes de proceder
if len(v6025_2) > 0:
    # Agregar columna notas
    v6025_2['notas'] = ""
    
    # Agregar v6025_2 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6025_2], ignore_index=True)

# Calcular s6025_2 (suma de Saldos y Saldos del mes pasado)
s6025_2 = abs(v6025_2['Saldos'].sum() + v6025_2['Saldos del mes pasado'].sum())

# 6030 PERÍODO ANTERIOR

In [53]:
# Filtrar y combinar subconjuntos
codigos = ['168005', '168010']
N3Subset = pd.concat([
    N3[N3['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear v6030_2 usando operaciones de pandas
v6030_2 = (N3Subset[['Cuenta alternativa', 'acumulado_mes', 'Arrastre saldos', 'Ejercicio']]
           .rename(columns={'acumulado_mes': 'Saldos', 
                           'Arrastre saldos': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
               **{'Unidad de captura': 6, 
                  'Subcuenta': '030'}
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                           'saldos_ant': 'Saldos del mes pasado'})
           .assign(
               **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                  'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
           )
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
           .reset_index(drop=True))

# Verificar si hay filas en v6030_2 antes de proceder
if len(v6030_2) > 0:
    # Agregar columna notas
    v6030_2['notas'] = ""
    
    # Agregar v6030_2 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6030_2], ignore_index=True)

# Calcular s6030_2 (suma de Saldos y Saldos del mes pasado)
s6030_2 = abs(v6030_2['Saldos'].sum() + v6030_2['Saldos del mes pasado'].sum())

# 6005 MES ACTUAL

In [54]:
# Filtrar y combinar subconjuntos
codigos = ['512105', '512135']
AÑO_ACTUALSubset = pd.concat([
    AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear a6005 usando operaciones de pandas
a6005 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
             **{'Unidad de captura': 6, 
                'Subcuenta': '005'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a6005 antes de proceder
if len(a6005) > 0:
    # Agregar columna notas
    a6005['notas'] = ""
    
    # Agregar a6005 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a6005], ignore_index=True)

# Calcular s6005
s6005 = sum(a6005['Saldos'])

#  6005  PASADO VENCIDO

In [55]:
# Filtrar y combinar subconjuntos
codigos = ['512105', '512135']
N1Subset = pd.concat([
    N1[N1['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear v6005_1 usando operaciones de pandas
v6005_1 = (N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
           .rename(columns={'Total Cuenta': 'Saldos', 
                           'Saldo mes pasado': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
               **{'Unidad de captura': 6, 
                  'Subcuenta': '005'}
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                           'saldos_ant': 'Saldos del mes pasado'})
           .assign(
               **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                  'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
           )
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
           .reset_index(drop=True))

# Verificar si hay filas en v6005_1 antes de proceder
if len(v6005_1) > 0:
    # Agregar columna notas
    v6005_1['notas'] = ""
    
    # Agregar v6005_1 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6005_1], ignore_index=True)

# Calcular s6005_1
s6005_1 = sum(v6005_1['Saldos'])

# 6005 #N-2 VENCIDO 

In [56]:
# Filtrar y combinar subconjuntos
codigos = ['512105', '512135']
N2Subset = pd.concat([
    N2[N2['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear v6005_2 usando operaciones de pandas
v6005_2 = (N2Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
           .rename(columns={'Total Cuenta': 'Saldos', 
                           'Saldo mes pasado': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
               **{'Unidad de captura': 6, 
                  'Subcuenta': '005'}
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                           'saldos_ant': 'Saldos del mes pasado'})
           .assign(
               **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                  'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
           )
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
           .reset_index(drop=True))

# Verificar si hay filas en v6005_2 antes de proceder
if len(v6005_2) > 0:
    # Agregar columna notas
    v6005_2['notas'] = ""
    
    # Agregar v6005_2 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6005_2], ignore_index=True)

# Calcular s6005_2
s6005_2 = sum(v6005_2['Saldos'])

# 6005  #N-3 VENCIDO

In [57]:
# Filtrar y combinar subconjuntos
codigos = ['512105', '512135']
N3Subset = pd.concat([
    N3[N3['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear v6005_3 usando operaciones de pandas
v6005_3 = (N3Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
           .rename(columns={'Total Cuenta': 'Saldos', 
                           'Saldo mes pasado': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
               **{'Unidad de captura': 6, 
                  'Subcuenta': '005'}
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                           'saldos_ant': 'Saldos del mes pasado'})
           .assign(
               **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                  'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
           )
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
           .reset_index(drop=True))

# Verificar si hay filas en v6005_3 antes de proceder
if len(v6005_3) > 0:
    # Agregar columna notas
    v6005_3['notas'] = ""
    
    # Agregar v6005_3 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6005_3], ignore_index=True)

# Calcular s6005_3
s6005_3 = sum(v6005_3['Saldos'])

# 6005 #N-3 MES CORRIDO 

In [58]:
# Filtrar y combinar subconjuntos
codigos = ['512105', '512135']
N3Subset = pd.concat([
    N3[N3['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear v6005_4 usando operaciones de pandas
v6005_4 = (N3Subset[['Cuenta alternativa', 'acumulado_mes', 'Saldo mes pasado', 'Ejercicio']]
           .rename(columns={'acumulado_mes': 'Saldos', 
                           'Saldo mes pasado': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
               **{'Unidad de captura': 6, 
                  'Subcuenta': '005'}
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                           'saldos_ant': 'Saldos del mes pasado'})
           .assign(
               **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                  'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
           )
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
           .reset_index(drop=True))

# Verificar si hay filas en v6005_4 antes de proceder
if len(v6005_4) > 0:
    # Agregar columna notas
    v6005_4['notas'] = ""
    
    # Agregar v6005_4 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6005_4], ignore_index=True)

# Calcular s6005_4 (valor absoluto negativo)
s6005_4 = -abs(v6005_4['Saldos'].sum())

# 6010

In [59]:
# Filtrar y combinar subconjuntos
codigos = ['515150', '515155', '515445', '515450']
AÑO_ACTUALSubset = pd.concat([
    AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear a6010 usando operaciones de pandas
a6010 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
             **{'Unidad de captura': 6, 
                'Subcuenta': '010'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a6010 antes de proceder
if len(a6010) > 0:
    # Agregar columna notas
    a6010['notas'] = ""
    
    # Agregar a6010 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a6010], ignore_index=True)

# Calcular s6010
s6010 = sum(a6010['Saldos'])

# 6010  PASADO VENCIDO

In [60]:
# Filtrar y combinar subconjuntos
codigos = ['515150', '515155', '515445', '515450']
N1Subset = pd.concat([
    N1[N1['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear v6010_1 usando operaciones de pandas
v6010_1 = (N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
           .rename(columns={'Total Cuenta': 'Saldos', 
                           'Saldo mes pasado': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
               **{'Unidad de captura': 6, 
                  'Subcuenta': '010'}
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                           'saldos_ant': 'Saldos del mes pasado'})
           .assign(
               **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                  'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
           )
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
           .reset_index(drop=True))

# Verificar si hay filas en v6010_1 antes de proceder
if len(v6010_1) > 0:
    # Agregar columna notas
    v6010_1['notas'] = ""
    
    # Agregar v6010_1 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6010_1], ignore_index=True)

# Calcular s6010_1
s6010_1 = abs(v6010_1['Saldos'].sum())

# 6010 #N-2 VENCIDO

In [61]:
# Filtrar y combinar subconjuntos
codigos = ['515150', '515155', '515445', '515450']
N2Subset = pd.concat([
    N2[N2['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear v6010_2 usando operaciones de pandas
v6010_2 = (N2Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
           .rename(columns={'Total Cuenta': 'Saldos', 
                           'Saldo mes pasado': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
               **{'Unidad de captura': 6, 
                  'Subcuenta': '010'}
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                           'saldos_ant': 'Saldos del mes pasado'})
           .assign(
               **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                  'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
           )
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
           .reset_index(drop=True))

# Verificar si hay filas en v6010_2 antes de proceder
if len(v6010_2) > 0:
    # Agregar columna notas
    v6010_2['notas'] = ""
    
    # Agregar v6010_2 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6010_2], ignore_index=True)

# Calcular s6010_2
s6010_2 = abs(v6010_2['Saldos'].sum())

# 6010   #N-3 VENCIDO

In [62]:
# Filtrar y combinar subconjuntos
codigos = ['515150', '515155', '515445', '515450']
N3Subset = pd.concat([
    N3[N3['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear v6010_3 usando operaciones de pandas
v6010_3 = (N3Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
           .rename(columns={'Total Cuenta': 'Saldos', 
                           'Saldo mes pasado': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
               **{'Unidad de captura': 6, 
                  'Subcuenta': '010'}
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                           'saldos_ant': 'Saldos del mes pasado'})
           .assign(
               **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                  'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
           )
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
           .reset_index(drop=True))

# Verificar si hay filas en v6010_3 antes de proceder
if len(v6010_3) > 0:
    # Agregar columna notas
    v6010_3['notas'] = ""
    
    # Agregar v6010_3 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6010_3], ignore_index=True)

# Calcular s6010_3
s6010_3 = abs(v6010_3['Saldos'].sum())

# 6010  #N-3 MES CORRIDO

In [63]:
# Filtrar y combinar subconjuntos
codigos = ['515150', '515155', '515445', '515450']
N3Subset = pd.concat([
    N3[N3['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear v6010_4 usando operaciones de pandas
v6010_4 = (N3Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
           .rename(columns={'Total Cuenta': 'Saldos', 
                           'Saldo mes pasado': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
               **{'Unidad de captura': 6, 
                  'Subcuenta': '010'}
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                           'saldos_ant': 'Saldos del mes pasado'})
           .assign(
               **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                  'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
           )
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
           .reset_index(drop=True))

# Verificar si hay filas en v6010_4 antes de proceder
if len(v6010_4) > 0:
    # Agregar columna notas
    v6010_4['notas'] = ""
    
    # Agregar v6010_4 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6010_4], ignore_index=True)

# Calcular s6010_4 (valor absoluto negativo)
s6010_4 = -abs(v6010_4['Saldos'].sum())

# 6055

In [64]:
# Filtrar y combinar subconjuntos
codigos = ['419530', '419525']
AÑO_ACTUALSubset = pd.concat([
    AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear a6055 usando operaciones de pandas
a6055 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
             **{'Unidad de captura': 6, 
                'Subcuenta': '055'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a6055 antes de proceder
if len(a6055) > 0:
    # Agregar columna notas
    a6055['notas'] = ""
    
    # Agregar a6055 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a6055], ignore_index=True)

# Calcular s6055
s6055 = abs(a6055['Saldos'].sum())

# PASADO VENCIDO 6055

In [65]:
# Filtrar y combinar subconjuntos
codigos = ['419530', '419525']
N1Subset = pd.concat([
    N1[N1['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear v6055_1 usando operaciones de pandas
v6055_1 = (N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
           .rename(columns={'Total Cuenta': 'Saldos', 
                           'Saldo mes pasado': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
               **{'Unidad de captura': 6, 
                  'Subcuenta': '055'}
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                           'saldos_ant': 'Saldos del mes pasado'})
           .assign(
               **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                  'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
           )
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
           .reset_index(drop=True))

# Verificar si hay filas en v6055_1 antes de proceder
if len(v6055_1) > 0:
    # Agregar columna notas
    v6055_1['notas'] = ""
    
    # Agregar v6055_1 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6055_1], ignore_index=True)

# Calcular s6055_1
s6055_1 = abs(v6055_1['Saldos'].sum())

#  #N-2 VENCIDO 

In [66]:
# Filtrar y combinar subconjuntos
codigos = ['419530', '419525']
N2Subset = pd.concat([
    N2[N2['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear v6055_2 usando operaciones de pandas
v6055_2 = (N2Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
           .rename(columns={'Total Cuenta': 'Saldos', 
                           'Saldo mes pasado': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
               **{'Unidad de captura': 6, 
                  'Subcuenta': '055'}
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                           'saldos_ant': 'Saldos del mes pasado'})
           .assign(
               **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                  'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
           )
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
           .reset_index(drop=True))

# Verificar si hay filas en v6055_2 antes de proceder
if len(v6055_2) > 0:
    # Agregar columna notas
    v6055_2['notas'] = ""
    
    # Agregar v6055_2 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6055_2], ignore_index=True)

# Calcular s6055_2
s6055_2 = abs(v6055_2['Saldos'].sum())

# N-3 VENCIDO 

In [67]:
# Filtrar y combinar subconjuntos
codigos = ['419530', '419525']
N3Subset = pd.concat([
    N3[N3['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear v6055_3 usando operaciones de pandas
v6055_3 = (N3Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
           .rename(columns={'Total Cuenta': 'Saldos', 
                           'Saldo mes pasado': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
               **{'Unidad de captura': 6, 
                  'Subcuenta': '055'}
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                           'saldos_ant': 'Saldos del mes pasado'})
           .assign(
               **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                  'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
           )
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
           .reset_index(drop=True))

# Verificar si hay filas en v6055_3 antes de proceder
if len(v6055_3) > 0:
    # Agregar columna notas
    v6055_3['notas'] = ""
    
    # Agregar v6055_3 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6055_3], ignore_index=True)

# Calcular s6055_3
s6055_3 = abs(v6055_3['Saldos'].sum())

# N-4 CORRIDO 

In [68]:
# Filtrar y combinar subconjuntos
codigos = ['419530', '419525']
N3Subset = pd.concat([
    N3[N3['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear v6055_4 usando operaciones de pandas
v6055_4 = (N3Subset[['Cuenta alternativa', 'acumulado_mes', 'Saldo mes pasado', 'Ejercicio']]
           .rename(columns={'acumulado_mes': 'Saldos', 
                           'Saldo mes pasado': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
               **{'Unidad de captura': 6, 
                  'Subcuenta': '055'}
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                           'saldos_ant': 'Saldos del mes pasado'})
           .assign(
               **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                  'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
           )
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
           .reset_index(drop=True))

# Verificar si hay filas en v6055_4 antes de proceder
if len(v6055_4) > 0:
    # Agregar columna notas
    v6055_4['notas'] = ""
    
    # Agregar v6055_4 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6055_4], ignore_index=True)

# Calcular s6055_4 (sin valor absoluto)
s6055_4 = v6055_4['Saldos'].sum()

#  SUMATORIAS

In [69]:
# Calcular los totales sumando las variables individuales
p6005 = s6005 + s6005_1 + s6005_2 + s6005_3 + s6005_4
p6010 = s6010 + s6010_1 + s6010_2 + s6010_3 + s6010_4
p6055 = s6055 + s6055_1 + s6055_2 + s6055_3 + s6055_4

# DATAFRAME

In [70]:
# Crear el primer dataframe con saldos y subcuentas
saldos06 = [round(p6005), round(p6010), round(s6015), round(s6020), 
            round(s6025_2), round(s6030_2), round(p6055)]
subcuentas06 = ['005', '010', '015', '020', '025', '030', '055']
unidad6998 = pd.DataFrame({
    'subcuentas06': subcuentas06,
    'saldos06': saldos06
})

# Redondear y tomar valor absoluto de saldos
unidad6998['saldos06'] = unidad6998['saldos06'].round(0).abs()

# Cálculos extra
v6060 = unidad6998['saldos06'].iloc[0:4].sum() - unidad6998['saldos06'].iloc[4:7].sum()
v6065 = v6060/3
v6070 = min(UVR * 25000000, v6065) * 0.27
v6075 = round(max(v6065 - UVR * 25000000, 0) * 0.24, 0)
v6115 = v6075 + v6070

# Crear segundo dataframe
subcuentas06_2 = ['060', '065', '070', '075', '115']
saldos06_2 = [v6060, v6065, v6070, v6075, v6115]
unidad6998_2 = pd.DataFrame({
    'subcuentas06': subcuentas06_2,
    'saldos06': saldos06_2
})

# Combinar los dataframes
unidad6998 = pd.concat([unidad6998, unidad6998_2], ignore_index=True)

# 7010

In [71]:
# Filtrar y combinar subconjuntos
codigos = ['414250', '414255', '414440', '414445']
AÑO_ACTUALSubset = pd.concat([
    AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear a7010 usando operaciones de pandas
a7010 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
             **{'Unidad de captura': 7, 
                'Subcuenta': '010'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a7010 antes de proceder
if len(a7010) > 0:
    # Agregar columna notas
    a7010['notas'] = ""
    
    # Agregar a7010 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a7010], ignore_index=True)

# Calcular s7010
s7010 = a7010['Saldos'].sum()

# 7010  PASADO VENCIDO 

In [72]:
# Filtrar y combinar subconjuntos
codigos = ['414250', '414255', '414440', '414445']
N1Subset = pd.concat([
    N1[N1['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear v7010_1 usando operaciones de pandas
v7010_1 = (N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
           .rename(columns={'Total Cuenta': 'Saldos', 
                           'Saldo mes pasado': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
               **{'Unidad de captura': 7, 
                  'Subcuenta': '010'}
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                           'saldos_ant': 'Saldos del mes pasado'})
           .assign(
               **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                  'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
           )
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
           .reset_index(drop=True))

# Verificar si hay filas en v7010_1 antes de proceder
if len(v7010_1) > 0:
    # Agregar columna notas
    v7010_1['notas'] = ""
    
    # Agregar v7010_1 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v7010_1], ignore_index=True)

# Calcular s7010_1
s7010_1 = v7010_1['Saldos'].sum()

# 7010  PASADO MES 

In [73]:
# Extraer las columnas necesarias
v7010_2 = N1Subset['acumulado_mes']
v7010_2ca = N1Subset['Cuenta alternativa']

# Calcular la suma absoluta
s7010_2 = abs(sum(v7010_2))

# Crear el DataFrame
v7010_2 = pd.DataFrame({
    'Cuenta alternativa': v7010_2ca,
    'saldos': v7010_2
})

# Extraer los primeros 6 dígitos de Cuenta alternativa
v7010_2['Cuenta'] = v7010_2['Cuenta alternativa'].astype(str).str[:6]

# Hacer el matching con CUIF
v7010_2['Nombre Cuenta'] = v7010_2['Cuenta'].map(dict(zip(CUIF['CUENTA'], CUIF['NOMBRE'])))

#   7005

In [74]:
# Filtrar y combinar subconjuntos
codigos = ['512105', '512135', '512150', '512155', '515445', '515450']
AÑO_ACTUALSubset = pd.concat([
    AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear a7005 usando operaciones de pandas
a7005 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
             **{'Unidad de captura': 7, 
                'Subcuenta': '005'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a7005 antes de proceder
if len(a7005) > 0:
    # Agregar columna notas
    a7005['notas'] = ""
    
    # Agregar a7005 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a7005], ignore_index=True)

# Calcular s7005
s7005 = a7005['Saldos'].sum()

#   7005 PASADO VENCIDO 

In [75]:
# Filtrar y combinar subconjuntos
codigos = ['512105', '512135', '512150', '512155', '515445', '515450']
N1Subset = pd.concat([
    N1[N1['6d'].astype(str).str.contains(codigo, na=False)]
    for codigo in codigos
])

# Crear v7005_1 usando operaciones de pandas
v7005_1 = (N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
           .rename(columns={'Total Cuenta': 'Saldos', 
                           'Saldo mes pasado': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
               **{'Unidad de captura': 7, 
                  'Subcuenta': '005'}
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                           'saldos_ant': 'Saldos del mes pasado'})
           .assign(
               **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                  'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
           )
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
           .reset_index(drop=True))

# Verificar si hay filas en v7005_1 antes de proceder
if len(v7005_1) > 0:
    # Agregar columna notas
    v7005_1['notas'] = ""
    
    # Agregar v7005_1 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v7005_1], ignore_index=True)

# Calcular s7005_1
s7005_1 = v7005_1['Saldos'].sum()

# PASADO MES

In [76]:
# Extraer las columnas necesarias
v7005_2 = N1Subset['acumulado_mes']
v7005_2ca = N1Subset['Cuenta alternativa']

# Calcular la suma absoluta (multiplicada por -1)
s7005_2 = abs(sum(v7005_2)) * -1

# Crear el DataFrame
v7005_2 = pd.DataFrame({
    'Cuenta alternativa': v7005_2ca,
    'saldos': v7005_2
})

# Extraer los primeros 6 dígitos de Cuenta alternativa
v7005_2['Cuenta'] = v7005_2['Cuenta alternativa'].astype(str).str[:6]

# Hacer el matching con CUIF
v7005_2['Nombre Cuenta'] = v7005_2['Cuenta'].map(dict(zip(CUIF['CUENTA'], CUIF['NOMBRE'])))

#  SUMATORIAS

In [77]:
# Calcular p7010
p7010 = s7010 + s7010_1 + s7010_2

# Calcular p7005
p7005 = s7005 + s7005_1 + s7005_2

# DATAFRAME 

In [78]:
# Crear el primer DataFrame
saldos07 = [p7005, p7010]
subcuentas07 = ['005', '010']

unidad7998 = pd.DataFrame({
    'subcuentas07': subcuentas07,
    'saldos07': saldos07
})

# Aplicar valor absoluto y redondeo
unidad7998['saldos07'] = abs(np.round(unidad7998['saldos07']))

# Calcular los valores manteniendo 4 decimales
v7015 = round((unidad7998['saldos07'].iloc[0] - unidad7998['saldos07'].iloc[1]) / unidad7998['saldos07'].iloc[0], 4)
v7020 = unidad7998['saldos07'].iloc[0]
v7025 = unidad7998['saldos07'].iloc[1]
v7030 = round((v7020 - v7025) / v7020, 4)
v7035 = 0
v7040 = 0
v7045 = 0
v7050 = round(max(v7015, 0.5), 4)
v7055 = round(max(v7030, 0.5), 4)
v7060 = 0

# Crear el segundo DataFrame
subcuenta_extra = ['015', '020', '025', '030', '035', '040', '045', '050', '055', '060']
saldos_extra = [v7015, v7020, v7025, v7030, v7035, v7040, v7045, v7050, v7055, v7060]

unidad7998_2 = pd.DataFrame({
    'subcuentas07': subcuenta_extra,
    'saldos07': saldos_extra
})

# Combinar los DataFrames
unidad7998 = pd.concat([unidad7998, unidad7998_2], ignore_index=True)

# Configurar el formato de visualización para mostrar 4 decimales
pd.set_option('display.float_format', lambda x: '{:.4f}'.format(x) if abs(x) < 1 else '{:.2f}'.format(x))

# Ajustar los valores según condición
decimales = ['015', '030', '050', '055']
unidad7998['saldos07'] = unidad7998.apply(
    lambda row: row['saldos07'] if row['subcuentas07'] in decimales else round(row['saldos07']),
    axis=1
)

# UNIDAD 9 

In [79]:
# Calcular los valores v9005, v9010 y v9998
v9005 = round(round(unidad7998.loc[9, 'saldos07'], 4) * unidad5998.loc[7, 'saldos05'])
v9010 = round(unidad7998.loc[9, 'saldos07'] * unidad6998.loc[11, 'saldos06'])
v9998 = round(max(v9005, v9010))

# Si estructura no existe o es una lista, crear un DataFrame vacío
if not isinstance(estructura, pd.DataFrame):
    estructura = pd.DataFrame(columns=['unidad', 'subcuenta', 'valor', 'descripcion'])

# Crear las nuevas filas
nuevas_filas = pd.DataFrame({
    'unidad': ['9', '9', '9'],
    'subcuenta': ['005', '010', '998'],
    'valor': [v9005, v9010, v9998],
    'descripcion': [
        "Este valor se calcula de tomar la subcuenta 050 de la unidad de captura 7 (O136) y la subcuenta 998 de la unidad de captura 5 (O100)",
        "Este valor se calcula de tomar la subcuenta 050 de la unidad de captura 7 (O136) y la subcuenta 115 de la unidad de captura 6 (O124)",
        "Este valor se calcula de tomar el valor maximo de las subcuentas 05 y 10 de la unidad de captura 9  "
    ]
})

# Agregar las nuevas filas a estructura
estructura = pd.concat([estructura, nuevas_filas], ignore_index=True)

# DATAFRAME

In [80]:
# Crear el DataFrame unidad9998
unidad9998 = pd.DataFrame({
    'Subcuenta': ['005', '010', '998'],
    'Saldos': [v9005, v9010, v9998]
})

# 10005

In [81]:
# Filtrar AÑO_ACTUAL
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].astype(str).str.contains('1105', na=False)]

# Crear a10005
a10005 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
          .rename(columns={'Total Cuenta': 'Saldos', 
                          'Saldo mes pasado': 'saldos_ant'})
          .assign(
              Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
              **{'Unidad de captura': 10, 
                 'Subcuenta': '005'}
          )
          .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
          .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                          'saldos_ant': 'Saldos del mes pasado'})
          .assign(
              **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                 'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
          )
          [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
            'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
            'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
          .reset_index(drop=True))

# Verificar si hay filas en a10005 antes de proceder
if len(a10005) > 0:
    # Agregar columna notas
    a10005['notas'] = ""
    
    # Agregar a10005 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a10005], ignore_index=True)

# Calcular s10005
s10005 = a10005['Saldos'].sum()

# 10010

In [82]:
# Filtrar AÑO_ACTUAL
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].astype(str).str.contains('1115', na=False)]

# Crear a10010
a10010 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
          .rename(columns={'Total Cuenta': 'Saldos', 
                          'Saldo mes pasado': 'saldos_ant'})
          .assign(
              Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
              **{'Unidad de captura': 10, 
                 'Subcuenta': '010'}
          )
          .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
          .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                          'saldos_ant': 'Saldos del mes pasado'})
          .assign(
              **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                 'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
          )
          [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
            'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
            'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
          .reset_index(drop=True))

# Verificar si hay filas en a10010 antes de proceder
if len(a10010) > 0:
    # Agregar columna notas
    a10010['notas'] = ""
    
    # Agregar a10010 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a10010], ignore_index=True)

# Calcular s10010
s10010 = a10010['Saldos'].sum()

# 10015

In [83]:
# Crear copia para evitar SettingWithCopyWarning
activos_c1 = activos[activos['CATEGORIA'] == 'I'].copy()

# Normalizar SUBCUENTA
activos_c1['SUBCUENTA'] = activos_c1['SUBCUENTA'].astype(str).str.strip().str.zfill(3)

# Filtrar por SUBCUENTA '015' y eliminar nulos como na.omit en R
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '015'].dropna(subset=['Valor Mercado ($)'])

# Sumar y redondear
v10015 = round(subsetactivos['Valor Mercado ($)'].sum(), 0)

# Crear fila
est4 = ["10", "015", v10015, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar a estructura
estructura.loc[len(estructura)] = est4


#  10020 

In [84]:
# Filtrar y copiar para evitar el warning
activos_c1 = activos[activos['CATEGORIA'] == 'I'].copy()

# Normalizar columnas sobre una copia segura
activos_c1['SUBCUENTA'] = activos_c1['SUBCUENTA'].astype(str).str.strip().str.zfill(3)

# Filtrar y eliminar nulos
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '020'].dropna()

# Sumar y redondear
v10020 = round(subsetactivos['Valor Mercado ($)'].sum(), 0)

# Crear la fila
est5 = ["10", "020", v10020, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar a estructura
estructura.loc[len(estructura)] = est5


# 10025

In [85]:
# Asegurarte de tener una copia del subconjunto para trabajar sin warnings
activos_c1 = activos[activos['CATEGORIA'] == 'I'].copy()

# Normalizar columnas (solo una vez si ya lo hiciste)
activos_c1['SUBCUENTA'] = activos_c1['SUBCUENTA'].astype(str).str.strip().str.zfill(3)

# Filtrar por SUBCUENTA '025' y eliminar nulos como na.omit en R
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '025'].dropna(subset=['Valor Mercado ($)'])

# Sumar y redondear a 0 decimales
v10025 = round(subsetactivos['Valor Mercado ($)'].sum(), 0)

# Crear la fila
est6 = ["10", "025", v10025, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar a estructura
estructura.loc[len(estructura)] = est6


#  10030 

In [86]:
# Asegurar que SUBCUENTA esté bien formateada (solo si no lo hiciste antes)
activos_c1['SUBCUENTA'] = activos_c1['SUBCUENTA'].astype(str).str.strip().str.zfill(3)

# Filtrar por SUBCUENTA '030' y eliminar nulos como na.omit
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '030'].dropna(subset=['Valor Mercado ($)'])

# Sumar y redondear
v10030 = round(subsetactivos['Valor Mercado ($)'].sum(), 0)

# Crear fila
est7 = ["10", "030", v10030, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar a estructura
estructura.loc[len(estructura)] = est7


#   10035

In [87]:
# Asegurarse de que SUBCUENTA esté normalizada (si no lo has hecho antes)
activos_c1['SUBCUENTA'] = activos_c1['SUBCUENTA'].astype(str).str.strip().str.zfill(3)

# Filtrar por SUBCUENTA '035' y eliminar nulos
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '035'].dropna(subset=['Valor Mercado ($)'])

# Sumar y redondear
v10035 = round(subsetactivos['Valor Mercado ($)'].sum(), 0)

# Crear fila
est8 = ["10", "035", v10035, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar a estructura
estructura.loc[len(estructura)] = est8


# 10040

In [88]:
# Asegurarse de que SUBCUENTA esté normalizada (si no lo hiciste antes)
activos_c1['SUBCUENTA'] = activos_c1['SUBCUENTA'].astype(str).str.strip().str.zfill(3)

# Filtrar por SUBCUENTA '040' y eliminar nulos
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '040'].dropna(subset=['Valor Mercado ($)'])

# Sumar y redondear
v10040 = round(subsetactivos['Valor Mercado ($)'].sum(), 0)

# Crear fila
est9 = ["10", "040", v10040, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar a estructura
estructura.loc[len(estructura)] = est9


#  10045

In [89]:
# Asegurarse de que SUBCUENTA esté bien formateada
activos_c1['SUBCUENTA'] = activos_c1['SUBCUENTA'].astype(str).str.strip().str.zfill(3)

# Filtrar sin eliminar filas con NaN
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '045']

# Sumar ignorando NaN (por defecto en pandas)
v10045 = round(subsetactivos['Valor Mercado ($)'].sum(skipna=True), 0)

# Crear fila
est10 = ["10", "045", v10045, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar a estructura
estructura.loc[len(estructura)] = est10

#  10050 

In [90]:
# Asegurarse de que SUBCUENTA esté bien formateada
activos_c1['SUBCUENTA'] = activos_c1['SUBCUENTA'].astype(str).str.strip().str.zfill(3)

# Filtrar por SUBCUENTA '050' y eliminar nulos
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '050'].dropna(subset=['Valor Mercado ($)'])

# Sumar y redondear
v10050 = round(subsetactivos['Valor Mercado ($)'].sum(), 0)

# Crear la fila
est11 = ["10", "050", v10050, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar a estructura
estructura.loc[len(estructura)] = est11

# 10055

In [91]:
# Asegurarse de que SUBCUENTA esté normalizada
activos_c1['SUBCUENTA'] = activos_c1['SUBCUENTA'].astype(str).str.strip().str.zfill(3)

# Filtrar por SUBCUENTA '055' y eliminar nulos como na.omit
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '055'].dropna(subset=['Valor Mercado ($)'])

# Sumar y redondear
v10055 = round(subsetactivos['Valor Mercado ($)'].sum(), 0)

# Crear la fila
est12 = ["10", "055", v10055, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar a estructura
estructura.loc[len(estructura)] = est12


# 10060

In [92]:
# Asegurar formato correcto de SUBCUENTA
activos_c1['SUBCUENTA'] = activos_c1['SUBCUENTA'].astype(str).str.strip().str.zfill(3)

# Filtrar por SUBCUENTA '060' y eliminar nulos
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '060'].dropna(subset=['Valor Mercado ($)'])

# Sumar y redondear
v10060 = round(subsetactivos['Valor Mercado ($)'].sum(), 0)

# Crear la fila
est13 = ["10", "060", v10060, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar a estructura
estructura.loc[len(estructura)] = est13

#  10065 

In [93]:
# Asegurarse de que SUBCUENTA esté bien formateada
activos_c1['SUBCUENTA'] = activos_c1['SUBCUENTA'].astype(str).str.strip().str.zfill(3)

# Filtrar por SUBCUENTA '065' y eliminar nulos como na.omit
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '065'].dropna(subset=['Valor Mercado ($)'])

# Sumar y redondear
v10065 = round(subsetactivos['Valor Mercado ($)'].sum(), 0)

# Crear la fila
est14 = ["10", "065", v10065, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar a estructura
estructura.loc[len(estructura)] = est14


# SUMATORIAS

In [94]:
# Calcular sumas (aunque en este caso son redundantes ya que s10005 y s10010 ya son sumas)
p10005 = s10005
p10010 = s10010

# Crear DataFrames individuales
v10005f = pd.DataFrame({
    'periodo': ['Mes actual'],
    'Saldos': [p10005],
    'Subcuenta': ['005']
})

v10010f = pd.DataFrame({
    'periodo': ['Mes actual'],
    'Saldos': [p10010],
    'Subcuenta': ['010']
})

# Combinar los DataFrames
captura10 = pd.concat([v10005f, v10010f], ignore_index=True)
captura10['Unidad de Captura'] = 10

# Crear unidad10998
unidad10998 = pd.DataFrame({
    'subcuentas10': ['005', '010'],
    'saldos10': [p10005, p10010]
})

# DATAFRAME

In [95]:
# Crear unidad10998_2 inicial
unidad10998_2 = pd.DataFrame({
    'Subcuenta': ['015', '020', '025', '030', '035', '040', '045',
                  '050', '055', '060', '065', '999']
})

# Calcular v10999
v10999 = sum([
    round(p10005), round(p10010), round(v10015), round(v10020), 
    round(v10025), round(v10030), round(v10035), round(v10040), 
    round(v10045), round(v10050), round(v10055), round(v10060), 
    round(v10065)
])

# Agregar columna de saldos a unidad10998_2
unidad10998_2['Saldos'] = [
    v10015, v10020, v10025, v10030, v10035,
    v10040, v10045, v10050, v10055, v10060, 
    v10065, v10999
]

# Renombrar columnas en unidad10998 si es necesario
unidad10998.columns = ['Subcuenta', 'Saldos']

# Combinar los DataFrames
unidad10998 = pd.concat([unidad10998, unidad10998_2], ignore_index=True)

# Redondear y tomar valor absoluto de Saldos
unidad10998['Saldos'] = unidad10998['Saldos'].round(0).abs()

# VALIDACIÓN REASEGURO

In [96]:
# Filtrar y sumar para cuenta 1677
reaseguro_1677_SAP = AÑO_ACTUAL[AÑO_ACTUAL['4d'].astype(str).str.contains('1677', na=False)]
reaseguro_1677_SAP_sum = reaseguro_1677_SAP['Total Cuenta'].sum()

# Filtrar y sumar para cuenta 1678
reaseguro_1678_SAP = AÑO_ACTUAL[AÑO_ACTUAL['4d'].astype(str).str.contains('1678', na=False)]
reaseguro_1678_SAP_sum = reaseguro_1678_SAP['Total Cuenta'].sum()

# Filtrar y sumar para cuenta 169515
reaseguro_169515_SAP = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('169515', na=False)]
reaseguro_169515_SAP_sum = reaseguro_169515_SAP['Total Cuenta'].sum()

# Filtrar y sumar para cuenta 1680
reaseguro_1680_SAP = AÑO_ACTUAL[AÑO_ACTUAL['4d'].astype(str).str.contains('1680', na=False)]
reaseguro_1680_SAP_sum = reaseguro_1680_SAP['Total Cuenta'].sum()

# CALIFICADO

In [97]:
# CALIFICADO_REASEGURO_1677
CALIFICADO_REASEGURO_1677 = pd.DataFrame({
    'total': [bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'CALIFICADO') & 
        (bd_reaseguros['Interior_Exterior'].str.strip() == 'Interior')
    ]['Cuenta_Corriente'].sum(skipna=True)]
})

# CALIFICADO_REASEGURO_1678
CALIFICADO_REASEGURO_1678 = pd.DataFrame({
    'total': [bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'CALIFICADO') & 
        (bd_reaseguros['Interior_Exterior'].str.strip() == 'Exterior')
    ]['Cuenta_Corriente'].sum(skipna=True)]
})

# CALIFICADO_REASEGURO_1680
CALIFICADO_REASEGURO_1680 = pd.DataFrame({
    'total': [bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'CALIFICADO') & 
        (bd_reaseguros['Interior_Exterior'].str.strip() == 'Exterior')
    ]['Activos_por_contingencias'].sum(skipna=True)]
})

# CALIFICADO_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO
CALIFICADO_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO = pd.DataFrame({
    'total': [-bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'CALIFICADO') & 
        (bd_reaseguros['Interior_Exterior'].str.strip() == 'Exterior')
    ]['Deterioro_Contingencia'].sum(skipna=True)]
})

# INTERIOR_REASEGURO_CALIFICACION
INTERIOR_REASEGURO_CALIFICACION = pd.DataFrame({
    'total': [-bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'CALIFICADO') & 
        (bd_reaseguros['Interior_Exterior'].str.strip() == 'Interior')
    ]['Deterioro'].sum(skipna=True)]
})

# EXTERIOR_REASEGURO_CALIFICACION
EXTERIOR_REASEGURO_CALIFICACION = pd.DataFrame({
    'total': [-bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'CALIFICADO') & 
        (bd_reaseguros['Interior_Exterior'].str.strip() == 'Exterior')
    ]['Deterioro'].sum(skipna=True)]
})

# CALIFICADO_REASEGURO_169515 inicial
CALIFICADO_REASEGURO_169515 = pd.DataFrame({
    'total': [bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'CALIFICADO') & 
        (bd_reaseguros['Interior_Exterior'].str.strip() == 'Exterior')
    ]['Cuenta_Corriente'].sum(skipna=True)]
})

# Actualizar CALIFICADO_REASEGURO_169515
CALIFICADO_REASEGURO_169515 = pd.DataFrame({
    'total': [INTERIOR_REASEGURO_CALIFICACION['total'].values[0] + 
              EXTERIOR_REASEGURO_CALIFICACION['total'].values[0]]
})

# SIN CALIFICACIÓN

In [98]:
# SIN_CALIFICACION_REASEGURO_1677
SIN_CALIFICACION_REASEGURO_1677 = pd.DataFrame({
    'total': [bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'SIN CALIFICACIÓN') & 
        (bd_reaseguros['Interior_Exterior'].str.strip() == 'Interior')
    ]['Cuenta_Corriente'].sum(skipna=True)]
})

# SIN_CALIFICACION_REASEGURO_1678
SIN_CALIFICACION_REASEGURO_1678 = pd.DataFrame({
    'total': [bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'SIN CALIFICACIÓN') & 
        (bd_reaseguros['Interior_Exterior'].str.strip() == 'Exterior')
    ]['Cuenta_Corriente'].sum(skipna=True)]
})

# SIN_CALIFICACION_REASEGURO_1680
SIN_CALIFICACION_REASEGURO_1680 = pd.DataFrame({
    'total': [bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'SIN CALIFICACIÓN')
    ]['Activos_por_contingencias'].sum(skipna=True)]
})

# SIN_CALIFICACION_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO
SIN_CALIFICACION_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO = pd.DataFrame({
    'total': [-bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'SIN CALIFICACIÓN')
    ]['Deterioro_Contingencia'].sum(skipna=True)]
})

# INTERIOR_REASEGURO_SIN_CALIFICACION
INTERIOR_REASEGURO_SIN_CALIFICACION = pd.DataFrame({
    'total': [-bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'SIN CALIFICACIÓN') & 
        (bd_reaseguros['Interior_Exterior'].str.strip() == 'Interior')
    ]['Deterioro'].sum(skipna=True)]
})

# EXTERIOR_REASEGURO_SIN_CALIFICACION
EXTERIOR_REASEGURO_SIN_CALIFICACION = pd.DataFrame({
    'total': [-bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'SIN CALIFICACIÓN') & 
        (bd_reaseguros['Interior_Exterior'].str.strip() == 'Exterior')
    ]['Deterioro'].sum(skipna=True)]
})

# SIN_CALIFICACION_REASEGURO_169515 inicial
temp_sin_calif_169515 = pd.DataFrame({
    'total': [bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'SIN CALIFICACIÓN') & 
        (bd_reaseguros['Interior_Exterior'].str.strip() == 'Exterior')
    ]['Cuenta_Corriente'].sum(skipna=True)]
})

# Actualizar SIN_CALIFICACION_REASEGURO_169515
SIN_CALIFICACION_REASEGURO_169515 = pd.DataFrame({
    'total': [INTERIOR_REASEGURO_SIN_CALIFICACION['total'].values[0] + 
              EXTERIOR_REASEGURO_SIN_CALIFICACION['total'].values[0]]
})

# TOTALES

In [99]:
# Calcular totales sumando valores calificados y sin calificación
TOTAL_CALIFICADO_REASEGURO_1677 = CALIFICADO_REASEGURO_1677 + SIN_CALIFICACION_REASEGURO_1677
TOTAL_CALIFICADO_REASEGURO_1678 = CALIFICADO_REASEGURO_1678 + SIN_CALIFICACION_REASEGURO_1678
TOTAL_CALIFICADO_REASEGURO_169515 = CALIFICADO_REASEGURO_169515 + SIN_CALIFICACION_REASEGURO_169515
TOTAL_CALIFICADO_REASEGURO_1680 = CALIFICADO_REASEGURO_1680 + SIN_CALIFICACION_REASEGURO_1680
TOTAL_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO = CALIFICADO_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO + SIN_CALIFICACION_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO
TOTAL_INTERIOR_REASEGURO_CALIFICACION = INTERIOR_REASEGURO_CALIFICACION + INTERIOR_REASEGURO_SIN_CALIFICACION
TOTAL_EXTERIOR_REASEGURO_CALIFICACION = EXTERIOR_REASEGURO_CALIFICACION + EXTERIOR_REASEGURO_SIN_CALIFICACION

# DIFERENCIAS

In [100]:
# Primero calculamos los totales
TOTAL_CALIFICADO_REASEGURO_1677 = pd.DataFrame({
    'total': [round(CALIFICADO_REASEGURO_1677['total'].iloc[0] + SIN_CALIFICACION_REASEGURO_1677['total'].iloc[0])]
})

TOTAL_CALIFICADO_REASEGURO_1678 = pd.DataFrame({
    'total': [round(CALIFICADO_REASEGURO_1678['total'].iloc[0] + SIN_CALIFICACION_REASEGURO_1678['total'].iloc[0])]
})

TOTAL_CALIFICADO_REASEGURO_169515 = pd.DataFrame({
    'total': [round(CALIFICADO_REASEGURO_169515['total'].iloc[0] + SIN_CALIFICACION_REASEGURO_169515['total'].iloc[0])]
})

TOTAL_CALIFICADO_REASEGURO_1680 = pd.DataFrame({
    'total': [round(CALIFICADO_REASEGURO_1680['total'].iloc[0] + SIN_CALIFICACION_REASEGURO_1680['total'].iloc[0])]
})

TOTAL_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO = pd.DataFrame({
    'total': [round(CALIFICADO_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO['total'].iloc[0] + 
              SIN_CALIFICACION_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO['total'].iloc[0])]
})

# Función auxiliar para obtener el valor numérico y redondearlo
def get_value(x):
    if isinstance(x, (pd.DataFrame, pd.Series)):
        return round(x['total'].iloc[0])
    return round(x)

# Ahora calculamos las diferencias
DIFERENCIA_REASEGURO1677 = pd.DataFrame({
    'total': [round(get_value(TOTAL_CALIFICADO_REASEGURO_1677) - get_value(reaseguro_1677_SAP_sum))]
})

DIFERENCIA_REASEGURO1678 = pd.DataFrame({
    'total': [round(get_value(TOTAL_CALIFICADO_REASEGURO_1678) - get_value(reaseguro_1678_SAP_sum))]
})

DIFERENCIA_REASEGURO169515 = pd.DataFrame({
    'total': [round(get_value(TOTAL_CALIFICADO_REASEGURO_169515) - get_value(reaseguro_169515_SAP_sum))]
})

DIFERENCIA_REASEGURO1680 = pd.DataFrame({
    'total': [round(get_value(TOTAL_CALIFICADO_REASEGURO_1680) - get_value(reaseguro_1680_SAP_sum) + 
              get_value(TOTAL_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO))]
})

# INFORMACIÓN REASEGURO

In [101]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

# Crear matriz vacía
INFORMACION_REASEGURO = np.empty((7, 7), dtype=object)

# Definir nombres de columnas
columnas = ["CUENTA", "NOMBRE", "SAP", "CALIFICADO", "SIN_CALIFICACION", "TOTAL", "DIFERENCIA"]

# Función auxiliar para obtener valores
def get_value(x):
    if isinstance(x, (pd.DataFrame, pd.Series)):
        return redondear_valor(x['total'].iloc[0])
    return redondear_valor(float(x)) if isinstance(x, (int, float)) else x

# Columna CUENTA (no aplicar redondeo)
INFORMACION_REASEGURO[0:4, 0] = [1677, 1678, 169515, 1680]

# Columna NOMBRE (no aplicar redondeo)
nombres = [
    "REASEGURADORES INTERIOR CUENTA CORRIENTE",
    "REASEGURADORES EXTERIOR CUENTA CORRIENTE",
    "DETERIORO CUENTAS POR COBRAR REASEGURADORES",
    "REASEGURADORES ACTIVOS POR CONTINGENCIAS",
    "DETERIORO ACTIVOS POR CONTINGENCIAS",
    "Interior",
    "Exterior"
]
INFORMACION_REASEGURO[:, 1] = nombres

# Columna SAP
INFORMACION_REASEGURO[0:4, 2] = [
    get_value(reaseguro_1677_SAP_sum),
    get_value(reaseguro_1678_SAP_sum),
    get_value(reaseguro_169515_SAP_sum),
    get_value(reaseguro_1680_SAP_sum)
]

# Columna CALIFICADO
INFORMACION_REASEGURO[:, 3] = [
    get_value(CALIFICADO_REASEGURO_1677),
    get_value(CALIFICADO_REASEGURO_1678),
    get_value(CALIFICADO_REASEGURO_169515),
    get_value(CALIFICADO_REASEGURO_1680),
    get_value(CALIFICADO_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO),
    get_value(INTERIOR_REASEGURO_CALIFICACION),
    get_value(EXTERIOR_REASEGURO_CALIFICACION)
]

# Columna SIN_CALIFICACION
INFORMACION_REASEGURO[:, 4] = [
    get_value(SIN_CALIFICACION_REASEGURO_1677),
    get_value(SIN_CALIFICACION_REASEGURO_1678),
    get_value(SIN_CALIFICACION_REASEGURO_169515),
    get_value(SIN_CALIFICACION_REASEGURO_1680),
    get_value(SIN_CALIFICACION_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO),
    get_value(INTERIOR_REASEGURO_SIN_CALIFICACION),
    get_value(EXTERIOR_REASEGURO_SIN_CALIFICACION)
]

# Columna TOTAL
INFORMACION_REASEGURO[:, 5] = [
    get_value(TOTAL_CALIFICADO_REASEGURO_1677),
    get_value(TOTAL_CALIFICADO_REASEGURO_1678),
    get_value(TOTAL_CALIFICADO_REASEGURO_169515),
    get_value(TOTAL_CALIFICADO_REASEGURO_1680),
    get_value(TOTAL_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO),
    get_value(TOTAL_INTERIOR_REASEGURO_CALIFICACION),
    get_value(TOTAL_EXTERIOR_REASEGURO_CALIFICACION)
]

# Columna DIFERENCIA
INFORMACION_REASEGURO[0:4, 6] = [
    get_value(DIFERENCIA_REASEGURO1677),
    get_value(DIFERENCIA_REASEGURO1678),
    get_value(DIFERENCIA_REASEGURO169515),
    get_value(DIFERENCIA_REASEGURO1680)
]

# Convertir a DataFrame para mejor visualización
INFORMACION_REASEGURO = pd.DataFrame(INFORMACION_REASEGURO, columns=columnas)

# VALIDACIÓN COASEGURO

# SAP

In [102]:
# Filtrar y sumar para coaseguro 1675
coaseguro_1675_SAP = AÑO_ACTUAL[AÑO_ACTUAL['4d'].astype(str).str.contains('1675')]
coaseguro_1675_SAP_sum = pd.DataFrame({
    'total': [coaseguro_1675_SAP['Total Cuenta'].sum()]
})

# Filtrar y sumar para coaseguro 1676
coaseguro_1676_SAP = AÑO_ACTUAL[AÑO_ACTUAL['4d'].astype(str).str.contains('1676')]
coaseguro_1676_SAP_sum = pd.DataFrame({
    'total': [coaseguro_1676_SAP['Total Cuenta'].sum()]
})

# Filtrar y sumar para coaseguro 169510
coaseguro_169510_SAP = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('169510')]
coaseguro_169510_SAP_sum = pd.DataFrame({
    'total': [coaseguro_169510_SAP['Total Cuenta'].sum()]
})

# CALIFICADO

In [103]:
# Para CALIFICADO_COASEGURO_1675
CALIFICADO_COASEGURO_1675 = pd.DataFrame({
    'total': [bd_COASEGURADORES_CUENTA_CORRIENTE_ACEPTADOS['Calificado'].sum(skipna=True)]
})

# Para CALIFICADO_COASEGURO_1676
CALIFICADO_COASEGURO_1676 = pd.DataFrame({
    'total': [bd_COASEGURADORES_CUENTA_CORRIENTE_CEDIDOS['Calificado'].sum(skipna=True)]
})

# Para CALIFICADO_COASEGURO_169510
CALIFICADO_COASEGURO_169510 = pd.DataFrame({
    'total': [bd_DETERIORO_CUENTAS_POR_COBRAR_ACTIVIDAD_ASEGURADORA_COASEGURADORES['Calificado'].sum(skipna=True)]
})

# NO CALIFICADO

In [104]:
# Para NO_CALIFICADO_COASEGURO_1675
NO_CALIFICADO_COASEGURO_1675 = pd.DataFrame({
    'total': [bd_COASEGURADORES_CUENTA_CORRIENTE_ACEPTADOS['No_Calificado '].sum(skipna=True)]
})

# Para NO_CALIFICADO_COASEGURO_1676
NO_CALIFICADO_COASEGURO_1676 = pd.DataFrame({
    'total': [bd_COASEGURADORES_CUENTA_CORRIENTE_CEDIDOS['No_Calificado '].sum(skipna=True)]
})

# Para NO_CALIFICADO_COASEGURO_169510
NO_CALIFICADO_COASEGURO_169510 = pd.DataFrame({
    'total': [bd_DETERIORO_CUENTAS_POR_COBRAR_ACTIVIDAD_ASEGURADORA_COASEGURADORES['No_Calificado '].sum(skipna=True)]
})

# TOTALES

In [105]:
# Calcular totales
TOTAL_COASEGURO_1675 = pd.DataFrame({
    'total': [CALIFICADO_COASEGURO_1675['total'].iloc[0] + NO_CALIFICADO_COASEGURO_1675['total'].iloc[0]]
})

TOTAL_COASEGURO_1676 = pd.DataFrame({
    'total': [CALIFICADO_COASEGURO_1676['total'].iloc[0] + NO_CALIFICADO_COASEGURO_1676['total'].iloc[0]]
})

TOTAL_COASEGURO_169510 = pd.DataFrame({
    'total': [CALIFICADO_COASEGURO_169510['total'].iloc[0] + NO_CALIFICADO_COASEGURO_169510['total'].iloc[0]]
})

# DIFERENCIAS

In [106]:
# Calcular diferencias con redondeo
def redondear_valor(x):
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

DIFERENCIA_COASEASEGURO1675 = pd.DataFrame({
    'total': [redondear_valor(TOTAL_COASEGURO_1675['total'].iloc[0] - coaseguro_1675_SAP_sum['total'].iloc[0])]
})

DIFERENCIA_COASEASEGURO1676 = pd.DataFrame({
    'total': [redondear_valor(TOTAL_COASEGURO_1676['total'].iloc[0] - coaseguro_1676_SAP_sum['total'].iloc[0])]
})

DIFERENCIA_COASEASEGURO169510 = pd.DataFrame({
    'total': [redondear_valor(TOTAL_COASEGURO_169510['total'].iloc[0] - coaseguro_169510_SAP_sum['total'].iloc[0])]
})

# INFORMACIÓN COASEGURO

In [107]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

# Crear matriz vacía
INFORMACION_COASEGURO = np.empty((3, 7), dtype=object)

# Definir nombres de columnas
columnas = ["CUENTA", "NOMBRE", "SAP", "CALIFICADO", "NO_CALIFICADO", "TOTAL", "DIFERENCIA"]

# Función auxiliar para obtener valores
def get_value(x):
    if isinstance(x, (pd.DataFrame, pd.Series)):
        return redondear_valor(x['total'].iloc[0])
    return redondear_valor(float(x)) if isinstance(x, (int, float)) else x

# Llenar la matriz
# Columna CUENTA (no aplicar redondeo a estos valores)
INFORMACION_COASEGURO[0:3, 0] = [1675, 1676, 169510]

# Columna NOMBRE (no aplicar redondeo a textos)
nombres = [
    "COASEGURADORES CUENTA CORRIENTE ACEPTADOS",
    "COASEGURADORES CUENTA CORRIENTE CEDIDOS",
    "DETERIORO CUENTAS POR COBRAR ACTIVIDAD ASEGURADORA COASEGURADORES"
]
INFORMACION_COASEGURO[:, 1] = nombres

# Columna SAP
INFORMACION_COASEGURO[:, 2] = [
    get_value(coaseguro_1675_SAP_sum),
    get_value(coaseguro_1676_SAP_sum),
    get_value(coaseguro_169510_SAP_sum)
]

# Columna CALIFICADO
INFORMACION_COASEGURO[:, 3] = [
    get_value(CALIFICADO_COASEGURO_1675),
    get_value(CALIFICADO_COASEGURO_1676),
    get_value(CALIFICADO_COASEGURO_169510)
]

# Columna NO_CALIFICADO
INFORMACION_COASEGURO[:, 4] = [
    get_value(NO_CALIFICADO_COASEGURO_1675),
    get_value(NO_CALIFICADO_COASEGURO_1676),
    get_value(NO_CALIFICADO_COASEGURO_169510)
]

# Columna TOTAL
INFORMACION_COASEGURO[:, 5] = [
    get_value(TOTAL_COASEGURO_1675),
    get_value(TOTAL_COASEGURO_1676),
    get_value(TOTAL_COASEGURO_169510)
]

# Columna DIFERENCIA
INFORMACION_COASEGURO[:, 6] = [
    get_value(DIFERENCIA_COASEASEGURO1675),
    get_value(DIFERENCIA_COASEASEGURO1676),
    get_value(DIFERENCIA_COASEASEGURO169510)
]

# Convertir a DataFrame para mejor visualización
INFORMACION_COASEGURO = pd.DataFrame(INFORMACION_COASEGURO, columns=columnas)

# 11005

In [108]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

# Filtrar activos por CATEGORIA == 'II'
activos_c1 = activos[activos['CATEGORIA'] == 'II']

# Filtrar por SUBCUENTA == '005'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '005']

# Calcular la suma del Valor Mercado y aplicar redondeo especial
v11005 = redondear_valor(subsetactivos['Valor Mercado ($)'].sum(skipna=True))

# Crear nueva fila para estructura
est15 = ["11", "005", v11005, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est15], columns=estructura.columns)], ignore_index=True)

# 11010

In [109]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

# Para subcuenta 010
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '010']
v11010 = redondear_valor(subsetactivos['Valor Mercado ($)'].sum(skipna=True))
est16 = ["11", "010", v11010, "Este valor se toma de la hoja Activos, como fuente de información"]
estructura = pd.concat([estructura, pd.DataFrame([est16], columns=estructura.columns)], ignore_index=True)

# 11015

In [110]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

# Para subcuenta 015
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '015']
v11015 = redondear_valor(subsetactivos['Valor Mercado ($)'].sum(skipna=True))
est17 = ["11", "015", v11015, "Este valor se toma de la hoja Activos, como fuente de información"]
estructura = pd.concat([estructura, pd.DataFrame([est17], columns=estructura.columns)], ignore_index=True)

# 11020

In [111]:
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

# Filtrar por SUBCUENTA == '020'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '020']

# Calcular la suma del Valor Mercado y aplicar redondeo especial
v11020 = redondear_valor(subsetactivos['Valor Mercado ($)'].sum(skipna=True))

# Crear nueva fila para estructura
est18 = ["11", "020", v11020, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est18], columns=estructura.columns)], ignore_index=True)


# 11040

In [112]:
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba
# Sumar valores de las filas 1-3 de la columna CALIFICADO y aplicar redondeo
v11040 = redondear_valor(INFORMACION_COASEGURO.iloc[0:3]['CALIFICADO'].astype(float).sum())

# Crear nueva fila para estructura
est19 = ["11", "040", v11040, "Este valor se toma de la hoja Coaseguro, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est19], columns=estructura.columns)], ignore_index=True)

# 11045

In [113]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

# Filtrar por SUBCUENTA == '045'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '045']

# Calcular la suma del Valor Mercado y aplicar redondeo especial
v11045 = redondear_valor(subsetactivos['Valor Mercado ($)'].sum(skipna=True))

# Crear nueva fila para estructura
est20 = ["11", "045", v11045, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est20], columns=estructura.columns)], ignore_index=True)

# 11050

In [114]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

# Filtrar por SUBCUENTA == '050'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '050']

# Calcular la suma del Valor Mercado y aplicar redondeo especial
v11050 = redondear_valor(subsetactivos['Valor Mercado ($)'].sum(skipna=True))

# Crear nueva fila para estructura
est21 = ["11", "050", v11050, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est21], columns=estructura.columns)], ignore_index=True)

# DATAFRAME

In [115]:
# Crear DataFrame inicial
unidad11998_2 = pd.DataFrame({
    'Subcuenta': ['005', '010', '015', '020', '040', '045', '050'],
    'Saldos': [v11005, v11010, v11015, v11020, v11040, v11045, v11050]
})

# Crear copia para unidad11998
unidad11998 = unidad11998_2.copy()

# Agregar fila con la suma total
suma_total = unidad11998['Saldos'].sum()
nueva_fila = pd.DataFrame({
    'Subcuenta': ['998'],
    'Saldos': [suma_total]
})

# Concatenar la nueva fila
unidad11998 = pd.concat([unidad11998, nueva_fila], ignore_index=True)

# Asegurar que Saldos sea numérico
unidad11998['Saldos'] = pd.to_numeric(unidad11998['Saldos'])

# UNIDAD 11 (PONDERADA) 

In [116]:
# Calcular los valores multiplicados por 0.015
v11005p = int(round(v11005 * 0.015, 0))
est22 = ["11", "005", v11005p, "Este valor se calcula de la subcuenta 005 de la unidad de captura 11 multiplicado por el 1,5%"]
estructura = pd.concat([estructura, pd.DataFrame([est22], columns=estructura.columns)], ignore_index=True)

# Para v11010p
v11010p = int(round(v11010 * 0.015, 0))
est23 = ["11", "010", v11010p, "Este valor se calcula de la subcuenta 010 de la unidad de captura 11 multiplicado por el 1,5%"]
estructura = pd.concat([estructura, pd.DataFrame([est23], columns=estructura.columns)], ignore_index=True)

# Para v11015p (ya tiene round pero ajustamos el orden)
v11015p = int(round(v11015 * 0.015, 0))
est24 = ["11", "015", v11015p, "Este valor se calcula de la subcuenta 015 de la unidad de captura 11 multiplicado por el 1,5%"]
estructura = pd.concat([estructura, pd.DataFrame([est24], columns=estructura.columns)], ignore_index=True)

# Para v11020p
v11020p = int(round(v11020 * 0.015, 0))
est25 = ["11", "020", v11020p, "Este valor se calcula de la subcuenta 020 de la unidad de captura 11 multiplicado por el 1,5%"]
estructura = pd.concat([estructura, pd.DataFrame([est25], columns=estructura.columns)], ignore_index=True)

# Para v11040p
v11040p = int(round(v11040 * 0.015, 0))
est26 = ["11", "040", v11040p, "Este valor se calcula de la subcuenta 040 de la unidad de captura 11 multiplicado por el 1,5%"]
estructura = pd.concat([estructura, pd.DataFrame([est26], columns=estructura.columns)], ignore_index=True)

# Para v11045p
v11045p = int(round(v11045 * 0.015, 0))
est27 = ["11", "045", v11045p, "Este valor se calcula de la subcuenta 045 de la unidad de captura 11 multiplicado por el 1,5%"]
estructura = pd.concat([estructura, pd.DataFrame([est27], columns=estructura.columns)], ignore_index=True)

# Para v11050p
v11050p = int(round(v11050 * 0.015, 0))
est28 = ["11", "050", v11050p, "Este valor se calcula de la subcuenta 045 de la unidad de captura 11 multiplicado por el 1,5%"]
estructura = pd.concat([estructura, pd.DataFrame([est28], columns=estructura.columns)], ignore_index=True)

# DATAFRAME

In [117]:
# Crear DataFrame inicial
unidad11998_p = pd.DataFrame({
    'Subcuenta': ['005', '010', '015', '020', '040', '045', '050'],
    'Saldos': [v11005p, v11010p, v11015p, v11020p, v11040p, v11045p, v11050p]
})

# Calcular la suma y redondear hacia arriba con ceiling
suma_total = np.ceil(unidad11998_p['Saldos'].sum())

# Agregar fila con la suma total
nueva_fila = pd.DataFrame({
    'Subcuenta': ['998'],
    'Saldos': [suma_total]
})

# Concatenar la nueva fila
unidad11998_p = pd.concat([unidad11998_p, nueva_fila], ignore_index=True)

# Asegurar que Saldos sea numérico
unidad11998_p['Saldos'] = pd.to_numeric(unidad11998_p['Saldos'])

# 12005

In [118]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

# Filtrar por CATEGORIA == 'III'
activos_c1 = activos[activos['CATEGORIA'] == 'III']

# Filtrar por SUBCUENTA == '005'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '005']

# Calcular la suma del Valor Mercado y aplicar redondeo especial
v12005 = redondear_valor(subsetactivos['Valor Mercado ($)'].sum(skipna=True))

# Crear nueva fila para estructura
est29 = ["12", "005", v12005, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est29], columns=estructura.columns)], ignore_index=True)

# 12010

In [119]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

# Filtrar por SUBCUENTA == '010'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '010']

# Calcular la suma del Valor Mercado y aplicar redondeo especial
v12010 = redondear_valor(subsetactivos['Valor Mercado ($)'].sum(skipna=True))

# Crear nueva fila para estructura
est30 = ["12", "010", v12010, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est30], columns=estructura.columns)], ignore_index=True)

# 12015

In [120]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

# Filtrar por SUBCUENTA == '015'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '015']

# Calcular la suma del Valor Mercado y aplicar redondeo especial
v12015 = redondear_valor(subsetactivos['Valor Mercado ($)'].sum(skipna=True))

# Crear nueva fila para estructura
est31 = ["12", "015", v12015, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est31], columns=estructura.columns)], ignore_index=True)

# 12020

In [121]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

# Filtrar por SUBCUENTA == '020'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '020']

# Calcular la suma del Valor Mercado y aplicar redondeo especial
v12020 = redondear_valor(subsetactivos['Valor Mercado ($)'].sum(skipna=True))

# Crear nueva fila para estructura
est32 = ["12", "020", v12020, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est32], columns=estructura.columns)], ignore_index=True)

# 12025

In [122]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

# Filtrar por SUBCUENTA == '025'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '025']

# Calcular la suma del Valor Mercado y aplicar redondeo especial
v12025 = redondear_valor(subsetactivos['Valor Mercado ($)'].sum(skipna=True))

# Crear nueva fila para estructura
est33 = ["12", "025", v12025, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est33], columns=estructura.columns)], ignore_index=True)

# 12030

In [123]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

# Filtrar por SUBCUENTA == '030'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '030']

# Calcular la suma del Valor Mercado y aplicar redondeo especial
v12030 = redondear_valor(subsetactivos['Valor Mercado ($)'].sum(skipna=True))

# Crear nueva fila para estructura
est34 = ["12", "030", v12030, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est34], columns=estructura.columns)], ignore_index=True)

# 12045

In [124]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

# Filtrar por SUBCUENTA == '045'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '045']

# Calcular la suma del Valor Mercado y aplicar redondeo especial
v12045 = redondear_valor(subsetactivos['Valor Mercado ($)'].sum(skipna=True))

# Crear nueva fila para estructura
est35 = ["12", "045", v12045, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est35], columns=estructura.columns)], ignore_index=True)

# 12050

In [125]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

# Filtrar por SUBCUENTA == '050'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '050']

# Calcular la suma del Valor Mercado y aplicar redondeo especial
v12050 = redondear_valor(subsetactivos['Valor Mercado ($)'].sum(skipna=True))

# Crear nueva fila para estructura
est36 = ["12", "050", v12050, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est36], columns=estructura.columns)], ignore_index=True)

# 12055

In [126]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

# Filtrar por SUBCUENTA == '055'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '055']

# Calcular la suma del Valor Mercado y aplicar redondeo especial
v12055 = redondear_valor(subsetactivos['Valor Mercado ($)'].sum(skipna=True))

# Crear nueva fila para estructura
est37 = ["12", "055", v12055, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est37], columns=estructura.columns)], ignore_index=True)

# 12060

In [127]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

# Filtrar por SUBCUENTA == '060'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '060']

# Calcular la suma del Valor Mercado y aplicar redondeo especial
v12060 = redondear_valor(subsetactivos['Valor Mercado ($)'].sum(skipna=True))

# Crear nueva fila para estructura
est38 = ["12", "060", v12060, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est38], columns=estructura.columns)], ignore_index=True)

# 12065

In [128]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

# Filtrar por SUBCUENTA == '065'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '065']

# Calcular la suma del Valor Mercado y aplicar redondeo especial
v12065 = redondear_valor(subsetactivos['Valor Mercado ($)'].sum(skipna=True))

# Crear nueva fila para estructura
est39 = ["12", "065", v12065, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est39], columns=estructura.columns)], ignore_index=True)

# 12070

In [129]:
# Obtener el valor de INFORMACION_REASEGURO
v12070 = float(INFORMACION_REASEGURO.iloc[3, 3])  # Ajustado a índices 0-based en Python

# Crear nueva fila para estructura
est40 = ["12", "070", v12070, "Este valor se toma de la hoja Reaseguro, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est40], columns=estructura.columns)], ignore_index=True)

# 12075

In [130]:
# Obtener el valor de INFORMACION_REASEGURO
v12075 = float(INFORMACION_REASEGURO.iloc[3, 4])  # Ajustado a índices 0-based en Python

# Crear nueva fila para estructura
est41 = ["12", "075", v12075, "Este valor se toma de la hoja Reaseguro, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est41], columns=estructura.columns)], ignore_index=True)

# 12080

In [131]:
# Obtener el valor de INFORMACION_REASEGURO
v12080 = float(INFORMACION_REASEGURO.iloc[1, 3])  # Ajustado a índices 0-based en Python

# Crear nueva fila para estructura
est42 = ["12", "080", v12080, "Este valor se toma de la hoja Reaseguro, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est42], columns=estructura.columns)], ignore_index=True)

# 12085

In [132]:
# Obtener el valor de INFORMACION_REASEGURO
v12085 = float(INFORMACION_REASEGURO.iloc[1, 4])  # Ajustado a índices 0-based en Python

# Crear nueva fila para estructura
est43 = ["12", "085", v12085, "Este valor se toma de la hoja Reaseguro, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est43], columns=estructura.columns)], ignore_index=True)

# 12125

In [133]:
# Función de redondeo especial para manejar NaN y valores no numéricos
def redondear_valor(x):
    try:
        if pd.isna(x) or x == '' or x is None:
            return 0
        val = float(x)
        return 0 if val >= 1e11 else round(val)  # Si el valor es muy grande, retorna 0
    except:
        return 0

# Obtener valores ajustando las referencias para que coincidan con Excel
v12125_1 = redondear_valor(Info.iloc[11, 2])  # Info!C13
v12125_2 = INFORMACION_COASEGURO.iloc[3:6, 5].apply(redondear_valor).sum()  # Coaseguro!F4:F6
v12125_3 = redondear_valor(INFORMACION_REASEGURO.iloc[3, 5])  # Reaseguro!F4

# Sumar todos los valores y redondear el resultado final
v12125 = round(sum([v12125_1, v12125_2, v12125_3]))

# Crear nueva fila para estructura
est44 = ["12", "0125", v12125, "Este valor se toma de la hoja Coaseguro, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est44], columns=estructura.columns)], ignore_index=True)


# DATAFRAME

In [134]:
# Crear el DataFrame inicial
unidad12998_2 = pd.DataFrame({
    'Subcuenta': ['005', '010', '015', '020', '025', '030', '045',
                  '050', '055', '060', '065', '070', '075', '080',
                  '085', '125']
})

# Agregar la columna de Saldos
unidad12998_2['Saldos'] = [v12005, v12010, v12015, v12020, v12025, v12030,
                           v12045, v12050, v12055, v12060, v12065, v12070,
                           v12075, v12080, v12085, v12125]

# Crear copia del DataFrame
unidad12998 = unidad12998_2.copy()

# Calcular la suma redondeada de los saldos
suma_total = round(unidad12998['Saldos'].sum())

# Agregar la fila con la suma total
nueva_fila = pd.DataFrame({
    'Subcuenta': ['998'],
    'Saldos': [suma_total]
})

# Concatenar la nueva fila
unidad12998 = pd.concat([unidad12998, nueva_fila], ignore_index=True)

# Asegurar que Saldos sea numérico
unidad12998['Saldos'] = pd.to_numeric(unidad12998['Saldos'])

# UNIDAD 12 (DETERIORADA) 

# 12070d

In [135]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    return np.ceil(x)  # Usar ceil para redondear hacia arriba

# Obtener el valor y convertirlo a negativo
v12070d = -float(INFORMACION_REASEGURO.iloc[4, 3])  # Ajustado a índices 0-based en Python

# Aplicar redondeo hacia arriba
v12070d = redondear_valor(v12070d)

# Crear nueva fila para estructura
est45 = ["12", "070", v12070d, "Este valor se toma de la hoja Reaseguro, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est45], columns=estructura.columns)], ignore_index=True)

# 12075d

In [136]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    return np.ceil(x)  # Usar ceil para redondear hacia arriba

# Obtener el valor y convertirlo a negativo
v12075d = -float(INFORMACION_REASEGURO.iloc[4, 4])  # Ajustado a índices 0-based en Python

# Aplicar redondeo hacia arriba
v12075d = redondear_valor(v12075d)

# Crear nueva fila para estructura
est46 = ["12", "075", v12075d, "Este valor se toma de la hoja Reaseguro, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est46], columns=estructura.columns)], ignore_index=True)

# 12080d

In [137]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    return np.ceil(x)  # Usar ceil para redondear hacia arriba

# Obtener el valor y convertirlo a negativo
v12080d = -float(INFORMACION_REASEGURO.iloc[6, 3])  # Ajustado a índices 0-based en Python

# Aplicar redondeo hacia arriba
v12080d = redondear_valor(v12080d)

# Crear nueva fila para estructura
est47 = ["12", "080", v12080d, "Este valor se toma de la hoja Reaseguro, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est47], columns=estructura.columns)], ignore_index=True)

# 12085d

In [138]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    return np.ceil(x)  # Usar ceil para redondear hacia arriba

# Obtener el valor y convertirlo a negativo
v12085d = -float(INFORMACION_REASEGURO.iloc[6, 4])  # Ajustado a índices 0-based en Python

# Aplicar redondeo hacia arriba
v12085d = redondear_valor(v12085d)

# Crear nueva fila para estructura
est48 = ["12", "085", v12085d, "Este valor se toma de la hoja Reaseguro, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est48], columns=estructura.columns)], ignore_index=True)

# 12125d

In [139]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    return np.ceil(x)  # Usar ceil para redondear hacia arriba

# Obtener el valor y convertirlo a negativo
v12125d = -float(INFORMACION_REASEGURO.iloc[5, 4])  # Ajustado a índices 0-based en Python

# Aplicar redondeo hacia arriba
v12125d = redondear_valor(v12125d)

# Crear nueva fila para estructura
est49 = ["12", "0125", v12125d, "Este valor se toma de la hoja Reaseguro, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est49], columns=estructura.columns)], ignore_index=True)

# DATAFRAME

In [140]:
# Crear listas para las cuentas y saldos
cuentas12 = ["070", "075", "080", "085", "125"]
saldos12 = [v12070d, v12075d, v12080d, v12085d, v12125d]

# Crear el DataFrame
unidad12998_d = pd.DataFrame({
    'Subcuenta': cuentas12,
    'Saldos': saldos12
})

# Calcular la suma de los saldos
suma_saldos = unidad12998_d['Saldos'].sum()

# Agregar la nueva fila con la suma
nueva_fila = pd.DataFrame({
    'Subcuenta': ['998'],
    'Saldos': [suma_saldos]
})

# Concatenar la nueva fila
unidad12998_d = pd.concat([unidad12998_d, nueva_fila], ignore_index=True)

# Asegurar que Saldos sea numérico
unidad12998_d['Saldos'] = pd.to_numeric(unidad12998_d['Saldos'])

# UNIDAD 12 (PONDERADA) 

In [141]:
# Calcular porcentajes y crear nuevas filas para estructura

# 005 - 1.5%
v12005p = int(round(v12005 * 0.015, 0))
est50 = ["12", "005", v12005p, "Este valor se calcula de la subcuenta 005 de la unidad de captura 12 multiplicado por el 1,5%"]
estructura = pd.concat([estructura, pd.DataFrame([est50], columns=estructura.columns)], ignore_index=True)

# 010 - 4.5%
v12010p = int(round(v12010 * 0.045, 0))
est51 = ["12", "010", v12010p, "Este valor se calcula de la subcuenta 010 de la unidad de captura 12 multiplicado por el 4,5%"]
estructura = pd.concat([estructura, pd.DataFrame([est51], columns=estructura.columns)], ignore_index=True)

# 015 - 8.5%
v12015p = int(round(v12015 * 0.085, 0))
est52 = ["12", "015", v12015p, "Este valor se calcula de la subcuenta 015 de la unidad de captura 12 multiplicado por el 8,5%"]
estructura = pd.concat([estructura, pd.DataFrame([est52], columns=estructura.columns)], ignore_index=True)

# 020 - 1.5%
v12020p = int(round(v12020 * 0.015, 0))
est53 = ["12", "020", v12020p, "Este valor se calcula de la subcuenta 020 de la unidad de captura 12 multiplicado por el 1,5%"]
estructura = pd.concat([estructura, pd.DataFrame([est53], columns=estructura.columns)], ignore_index=True)

# 025 - 4.5%
v12025p = int(round(v12025 * 0.045, 0))
est54 = ["12", "025", v12025p, "Este valor se calcula de la subcuenta 025 de la unidad de captura 12 multiplicado por el 4,5%"]
estructura = pd.concat([estructura, pd.DataFrame([est54], columns=estructura.columns)], ignore_index=True)

# 030 - 8.5%
v12030p = int(round(v12030 * 0.085, 0))
est55 = ["12", "030", v12030p, "Este valor se calcula de la subcuenta 030 de la unidad de captura 12 multiplicado por el 8,5%"]
estructura = pd.concat([estructura, pd.DataFrame([est55], columns=estructura.columns)], ignore_index=True)

# 045 - 1.5%
v12045p = int(round(v12045 * 0.015, 0))
est56 = ["12", "045", v12045p, "Este valor se calcula de la subcuenta 045 de la unidad de captura 12 multiplicado por el 1,5%"]
estructura = pd.concat([estructura, pd.DataFrame([est56], columns=estructura.columns)], ignore_index=True)

# 050 - 4.5%
v12050p = int(round(v12050 * 0.045, 0))
est57 = ["12", "050", v12050p, "Este valor se calcula de la subcuenta 050 de la unidad de captura 12 multiplicado por el 4,5%"]
estructura = pd.concat([estructura, pd.DataFrame([est57], columns=estructura.columns)], ignore_index=True)

# 055 - 8.5%
v12055p = int(round(v12055 * 0.085, 0))
est58 = ["12", "050", v12055p, "Este valor se calcula de la subcuenta 055 de la unidad de captura 12 multiplicado por el 8,5%"]
estructura = pd.concat([estructura, pd.DataFrame([est58], columns=estructura.columns)], ignore_index=True)

# 060 - 4.5%
v12060p = int(round(v12060 * 0.045, 0))
est59 = ["12", "060", v12060p, "Este valor se calcula de la subcuenta 060 de la unidad de captura 12 multiplicado por el 4,5%"]
estructura = pd.concat([estructura, pd.DataFrame([est59], columns=estructura.columns)], ignore_index=True)

# 065 - 4.5%
v12065p = int(round(v12065 * 0.045, 0))  # Cambiado de np.ceil a round para consistencia
est60 = ["12", "065", v12065p, "Este valor se calcula de la subcuenta 065 de la unidad de captura 12 multiplicado por el 4,5%"]
estructura = pd.concat([estructura, pd.DataFrame([est60], columns=estructura.columns)], ignore_index=True)

#  12070p 

In [142]:
# Filtrar el DataFrame de reaseguros para obtener solo los CALIFICADOS
reasegurosubset = bd_reaseguros[bd_reaseguros['Calificado'] == 'CALIFICADO']

# Calcular la suma de Ponderación_Activo y aplicar round en lugar de ceiling
v12070p = int(round(reasegurosubset['Ponderación_Activo'].astype(float).sum(skipna=True), 0))

# Crear nueva fila para estructura
est61 = ["12", "070", v12070p, "Este valor se toma de la hoja Reaseguro, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est61], columns=estructura.columns)], ignore_index=True)

# 12075p

In [143]:
# Calcular v12075p usando los valores de los DataFrames con redondeo
v12075p = int(round((float(unidad12998.iloc[12]['Saldos']) - float(unidad12998_d.iloc[1]['Saldos'])) * 0.085, 0))

# Crear nueva fila para estructura
est62 = ["12", "075", v12075p, "Este valor se calcula de la subcuenta 075 de la unidad de captura 12 multiplicado por el 8,5%"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est62], columns=estructura.columns)], ignore_index=True)

# 12080p

In [144]:
# Ajustamos el código asumiendo que el nombre de la columna podría ser diferente
# Por ejemplo, podría ser "Interior_Exterior" o "Ubicacion" u otro nombre
v12080p = int(round(
    bd_reaseguros[
        (bd_reaseguros['Calificado'] == 'CALIFICADO')
    ]['Ponderación_cta_cte'].apply(lambda x: float(x) if pd.notna(x) else 0).sum(),
    0
))

# Crear nueva fila para estructura
est63 = ["12", "080", v12080p, "Este valor se toma de la hoja Reaseguro, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est63], columns=estructura.columns)], ignore_index=True)

# 12085p

In [145]:
# Calcular v12085p con redondeo
v12085p = int(round(
    (float(unidad12998.iloc[14]['Saldos']) - float(unidad12998_d.iloc[3]['Saldos'])) * 0.085,
    0
))

# Crear nueva fila para estructura
est64 = ["12", "085", v12085p, "Este valor se calcula de la subcuenta 085 de la unidad de captura 12 multiplicado por el 8,5%"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est64], columns=estructura.columns)], ignore_index=True)

# 12125p

In [146]:
# Calcular v12125p con redondeo
v12125p = int(round(
    (float(unidad12998.iloc[15]['Saldos']) - float(unidad12998_d.iloc[4]['Saldos'])) * 0.085,
    0
))

# Crear nueva fila para estructura
est65 = ["12", "125", v12125p, "Este valor se calcula de la subcuenta 125 de la unidad de captura 12 multiplicado por el 8,5%"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est65], columns=estructura.columns)], ignore_index=True)

# DATAFRAME

In [147]:
# Crear DataFrame unidad12998_p
subcuentas = ['005', '010', '015', '020', '025', '030', '045',
              '050', '055', '060', '065', '070', '075', '080', 
              '085', '125']

saldos = [v12005p, v12010p, v12015p, v12020p, v12025p, v12030p,
          v12045p, v12050p, v12055p, v12060p, v12065p, v12070p,
          v12075p, v12080p, v12085p, v12125p]

unidad12998_p = pd.DataFrame({
    'Subcuenta': subcuentas,
    'Saldos': saldos
})

# Calcular la suma de saldos
suma_saldos = unidad12998_p['Saldos'].sum()

# Agregar la fila con la suma
nueva_fila = pd.DataFrame({
    'Subcuenta': ['998'],
    'Saldos': [suma_saldos]
})

# Concatenar la nueva fila
unidad12998_p = pd.concat([unidad12998_p, nueva_fila], ignore_index=True)

# Asegurar que Saldos sea numérico
unidad12998_p['Saldos'] = pd.to_numeric(unidad12998_p['Saldos'])

# UNIDAD 13

# v13005


In [148]:
# Obtener el valor usando el nombre correcto de la columna y manejar NaN
valor = Info.iloc[26]['Unnamed: 2']
v13005 = float(0 if pd.isna(valor) else valor)

# Crear nueva fila para estructura
est66 = ["13", "005", v12080p, "Este valor se toma de la hoja Información, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est66], columns=estructura.columns)], ignore_index=True)

# v13025

In [149]:
# Obtener el valor usando el nombre correcto de la columna y manejar NaN
valor = Info.iloc[28]['Unnamed: 2']
v13025 = float(0 if pd.isna(valor) else valor)

# Crear nueva fila para estructura
est67 = ["13", "025", v13025, "Este valor se toma de la hoja Información, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est67], columns=estructura.columns)], ignore_index=True)

# DATAFRAME

In [150]:
# Crear listas para subcuentas y saldos
cuentas13 = ['005', '025']
saldos13 = [v13005, v13025]

# Crear DataFrame
unidad13998 = pd.DataFrame({
    'Subcuenta': cuentas13,
    'Saldos': saldos13
})

# UNIDAD 14

In [151]:
# Obtener el valor usando el nombre correcto de la columna y manejar NaN
valor = Info.iloc[22]['Unnamed: 2']
# Manejar NaN y aplicar redondeo
if pd.isna(valor):
    v14005 = float(0)
else:
    v14005 = round(float(valor))  # Esto redondeará automáticamente según regla .5

# Crear nueva fila para estructura con el número correcto de columnas
est68 = ["14", "005", v14005, "Este valor se toma de la hoja Información, como fuente de información"]
# Añadir valores None para las columnas restantes
est68.extend([None] * (len(estructura.columns) - len(est68)))

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est68], columns=estructura.columns)], ignore_index=True)

# 14010

In [152]:
# Asumiendo que unidad9998 es un DataFrame
# Convertir el valor a numérico
v14010 = float(unidad9998['Saldos'].iloc[2])  # En Python los índices empiezan en 0, por eso usamos 2 en lugar de 3

# Crear el vector est69
est69 = ["14", "010", v14010, "Este valor se calcula de los totales de las subcuentas 998 de la unidad de captura 9"]

# Si estructura es un DataFrame, usar concat
estructura = pd.concat([estructura, pd.DataFrame([est69], columns=estructura.columns)], ignore_index=True)

# 14015

In [153]:
# Obtener valores y manejar NaN
valor1 = unidad11998_p.iloc[7]['Saldos']   # índice 7 para la fila 8
valor2 = unidad12998_p.iloc[16]['Saldos']  # índice 16 para la fila 17

# Sumar valores, manejando NaN
v14015 = float(0 if pd.isna(valor1) else valor1) + float(0 if pd.isna(valor2) else valor2)

# Crear nueva fila para estructura
est70 = ["14", "015", v14015, "Este valor se calcula de los totales de las subcuentas 998 de las unidades de captura 10,11 y 12"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est70], columns=estructura.columns)], ignore_index=True)

# 14020

In [154]:
v14020 = 1

# 14025

In [155]:
v14025 = 1

# 14030

In [156]:
v14030 = 1

# 14035

In [157]:
# Cálculo de v14035 y truncamiento a entero
v14035 = int(np.sqrt((v14010**2) + 
                    2*(v14010*v14015*v14025) + 
                    2*(v14010*v14005*v14020) + 
                    2*(v14015*v14005*v14030) + 
                    (v14015**2) + 
                    (v14005**2)))

# Creación del vector est71
est71 = ["14", "035", int(v14015), "Este valor se calcula sumando los demás valores de la unidad de captura 14"]

# Si estructura es un DataFrame, asegurar que est71 tenga el número correcto de columnas
est71.extend([None] * (len(estructura.columns) - len(est71)))

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est71], columns=estructura.columns)], ignore_index=True)

# 14040

In [158]:
# Cálculo de v14040
v14040 = v13005 * v13025  # Eliminado sum() ya que v13005 es un solo número

# Crear el vector est72
est72 = ["14", "040", v14040, "Este valor se calcula sumando las subcuentas 015 y 020 de la unidad de captura 13"]

# Añadir valores None para las columnas restantes si es necesario
est72.extend([None] * (len(estructura.columns) - len(est72)))

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est72], columns=estructura.columns)], ignore_index=True)

# 14998

In [159]:
# Sumar v14035 y v14040
v14998 = v14035 + v14040

# DATAFRAME

In [160]:
# Crear las listas de cuentas y saldos
cuentas14 = ['005', '010', '015', '020', '025', '030', '035', '040', '998']
saldos14 = [int(v14005), int(v14010), int(v14015), int(v14020), int(v14025), 
            int(v14030), int(v14035), int(v14040), int(v14998)]

# Crear el DataFrame
unidad14998 = pd.DataFrame({
    'Subcuenta': cuentas14,
    'Saldos': saldos14
})

# UNIDAD 15

In [161]:
# Cálculo de v15005 usando el último elemento de cada DataFrame y convirtiéndolo a entero
v15005 = int(unidad1998['Saldos'].iloc[-1] - unidad2998['Saldos'].iloc[-1])

# Crear el vector est73
est73 = ["15", "005", v15005, "Este valor se calcula tomando el total del subcuenta 998 de la unidad de captura 01 y restandole el total de la subcuenta 998 de la unidad de captura 02"]

# Añadir valores None para las columnas restantes si es necesario
est73.extend([None] * (len(estructura.columns) - len(est73)))

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est73], columns=estructura.columns)], ignore_index=True)

# 15010

In [162]:
v15010 = 0

# 15015

In [163]:
v15015 = min(unidad4998.iloc[0]['Saldos'], unidad14998.iloc[8]['Saldos'] * 0.15)

# 15020

In [164]:
v15020 = 0

# 15998

In [165]:
# Calcular v15998
v15998 = v15005 + v15010 + v15015 - v15020

# Crear nueva fila para estructura
est74 = ["15", "010", v15998, "Este valor se calcula sumando los demás valores de la unidad de captura 15"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est74], columns=estructura.columns)], ignore_index=True)

# DATAFRAME

In [166]:
# Crear listas con los datos
cuentas15 = ["005", "010", "015", "998"]
saldos15 = [v15005, v15010, v15015, v15998]

# Crear DataFrame
unidad15998 = pd.DataFrame({
    'Subcuenta': cuentas15,
    'Saldos': saldos15
})

# UNIDAD 16

In [167]:
# Calcular v16005
v16005 = v15998 - v14998

# Crear nueva fila para estructura
est75 = ["15", "005", v16005, "Este valor se calcula sumando los totales de la unidad de captura 998 de la unidades de captura 14 y 15"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est75], columns=estructura.columns)], ignore_index=True)

# 16010

In [168]:
v16010 = 0

# 16015

In [169]:
v16015 = v16010 * 0.035

# DATAFRAME

In [170]:
# Crear listas con los datos
cuentas16 = ["005", "010", "015"]
saldos16 = [v16005, v16010, v16015]

# Crear DataFrame
unidad16998 = pd.DataFrame({
    'Subcuenta': cuentas16,
    'Saldos': saldos16
})

# IMPRIMIR FORMATO 407

In [171]:
# Limpiar nombres de columnas (eliminar espacios extra)
BALANCE.columns = BALANCE.columns.str.strip()

# Buscar la columna que contenga 'Pos_balance' y 'Total_periodos_de_informe' de forma flexible
columna_pos_balance = next((col for col in BALANCE.columns if 'Pos_balance' in col), None)
columna_total_periodos = next((col for col in BALANCE.columns if 'Total_periodos_de_informe' in col), None)
columna_texto_pos = next((col for col in BALANCE.columns if 'Texto_ posicion_balance_PyG' in col), None)

if not columna_pos_balance or not columna_total_periodos or not columna_texto_pos:
    raise ValueError("No se encontraron las columnas necesarias en BALANCE.")

# Filtrar por 'Pos_balance' que contenga '865'
BalanceSubset = BALANCE[BALANCE[columna_pos_balance].astype(str).str.contains('865', na=False)]

# Filtrar por 'Texto_ posicion_balance_PyG' que sea 'RESULTADOS DEL EJERCICIO UTILIDAD'
BalanceSubset = BalanceSubset[BalanceSubset[columna_texto_pos].str.strip().str.lower() == 'resultados del ejercicio utilidad']

# Obtener el primer valor de 'Total_periodos_de_informe' si existe
Utilidad_del_ejercicio = BalanceSubset[columna_total_periodos].iloc[0] if not BalanceSubset.empty else None

# Escribir en Excel en la celda específica
hoja = wb['Hoja1']
hoja.cell(row=23, column=9, value=Utilidad_del_ejercicio)

<Cell 'Hoja1'.I23>

# PATRIMONIO BÁSICO ORDINARIO ANTES DE DEDUCCIONES

In [172]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad1998 existe
if 'unidad1998' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 23, "010": 24, "015": 25, "020": 26, "025": 27,
        "030": 28, "035": 29, "040": 30, "045": 31, "050": 32,
        "055": 33, "060": 34, "065": 35, "070": 36, "075": 37,
        "080": 38, "085": 39, "090": 40, "095": 41, "100": 42,
        "105": 43, "110": 44, "998": 45
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad1998[unidad1998['Subcuenta'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['Saldos'].iloc[0]
                # Redondear el valor
                valor = round(valor, 0)
            else:
                valor = 0  # valor por defecto cuando no hay datos
                
            # Escribir en Excel
            wb['Hoja1'].cell(row=row, column=4, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=4, value=0)  # valor por defecto en caso de error

# DEDUCCIONES DEL PATRIMONIO BÁSICO ORDINARIO 

In [173]:
# Verificar si "unidad2998" está en la lista de consultas
if "unidad2998" in consultas:

    # Función auxiliar para procesar cada subcuenta
    def procesar_subcuenta(subcuenta, fila, desde_info=False, info_fila=None, info_col=None, verbose=False):
        try:
            sheet = wb["Hoja1"]
            
            if desde_info and info_fila is not None and info_col is not None:
                # Extraer valor desde Info
                try:
                    valor_str = str(Info.iloc[info_fila, info_col])
                    valor_str = valor_str.replace(',', '')  # Eliminar comas
                    valor = float(valor_str)
                    valor = round(valor, 0)
                    if not pd.isna(valor):  # Verificar que no sea NaN
                        sheet.cell(row=fila, column=4, value=int(valor))
                        if verbose:
                            print(f"Subcuenta {subcuenta} (desde Info): {int(valor)}")
                    else:
                        if verbose:
                            print(f"El valor para subcuenta {subcuenta} desde Info es NaN")
                except Exception as e:
                    if verbose:
                        print(f"Error al procesar subcuenta {subcuenta} desde Info: {e}")
                    # Intentar obtener desde unidad2998 como fallback
                    try:
                        if any(unidad2998['Subcuenta'] == subcuenta):
                            valor = unidad2998.loc[unidad2998['Subcuenta'] == subcuenta, 'Saldos'].iloc[0]
                            valor = round(valor, 0)
                            if not pd.isna(valor):  # Verificar que no sea NaN
                                sheet.cell(row=fila, column=4, value=int(valor))
                                if verbose:
                                    print(f"Subcuenta {subcuenta} (desde unidad2998): {int(valor)}")
                            else:
                                if verbose:
                                    print(f"El valor para subcuenta {subcuenta} desde unidad2998 es NaN")
                    except:
                        if verbose:
                            print(f"No se pudo obtener valor para subcuenta {subcuenta} de ninguna fuente")
            else:
                # Extraer valor desde unidad2998
                if any(unidad2998['Subcuenta'] == subcuenta):
                    valor = unidad2998.loc[unidad2998['Subcuenta'] == subcuenta, 'Saldos'].iloc[0]
                    valor = round(valor, 0)
                    if not pd.isna(valor):  # Verificar que no sea NaN
                        sheet.cell(row=fila, column=4, value=int(valor))
                        if verbose:
                            print(f"Subcuenta {subcuenta}: {int(valor)}")
                    else:
                        if verbose:
                            print(f"El valor para subcuenta {subcuenta} es NaN")
                else:
                    if verbose:
                        print(f"No se encontró subcuenta {subcuenta}")
        except Exception as e:
            if verbose:
                print(f"Error general al procesar subcuenta {subcuenta}: {e}")

    # Procesar todas las subcuentas sin imprimir mensajes (verbose=False)
    procesar_subcuenta('005', 48, verbose=False)
    procesar_subcuenta('010', 49, verbose=False)
    procesar_subcuenta('015', 50, verbose=False)
    procesar_subcuenta('020', 51, verbose=False)
    procesar_subcuenta('025', 52, verbose=False)
    procesar_subcuenta('030', 53, verbose=False)
    procesar_subcuenta('035', 54, verbose=False)
    procesar_subcuenta('040', 55, verbose=False)
    
    # Para 045, usar Info[12,2] (ajustado a indexación de Python)
    procesar_subcuenta('045', 56, desde_info=True, info_fila=11, info_col=1, verbose=False)
    
    procesar_subcuenta('050', 57, verbose=False)
    procesar_subcuenta('055', 58, verbose=False)
    procesar_subcuenta('060', 59, verbose=False)
    procesar_subcuenta('065', 60, verbose=False)
    procesar_subcuenta('070', 61, verbose=False)
    procesar_subcuenta('075', 62, verbose=False)
    procesar_subcuenta('998', 63, verbose=False)

# PATRIMONIO BÁSICO ADICIONAL (PBA)

In [174]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad3998 existe
if 'unidad3998' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 66, "010": 67, "015": 68, "020": 69, "025": 70,
        "030": 71, "035": 72, "040": 73, "045": 74, "050": 75,
        "999": 76
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad3998[unidad3998['Subcuenta'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['Saldos'].iloc[0]
                # Redondear el valor
                valor = round(valor, 0)
            else:
                valor = 0
                
            # Escribir en Excel
            wb['Hoja1'].cell(row=row, column=4, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=4, value=0)

# PATRIMONIO ADICIONAL (PA) 

In [175]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad4998 existe
if 'unidad4998' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 79, "010": 80, "015": 81, "020": 82,
        "025": 83, "030": 84, "999": 85
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad4998[unidad4998['Subcuenta'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['Saldos'].iloc[0]
                # Redondear el valor
                valor = round(valor, 0)
            else:
                valor = 0
                
            # Escribir en Excel
            wb['Hoja1'].cell(row=row, column=4, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=4, value=0)

# PRIMAS

In [176]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad5998 existe
if 'unidad5998' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 93, "010": 94, "015": 95, "020": 96,
        "025": 97, "030": 98, "035": 99, "998": 100
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad5998[unidad5998['subcuentas05'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['saldos05'].iloc[0]
                # Redondear el valor
                valor = round(valor, 0)
            else:
                valor = 0
                
            # Escribir en Excel
            wb['Hoja1'].cell(row=row, column=15, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=15, value=0)

# SINIESTROS

In [177]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad6998 existe
if 'unidad6998' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 102, "010": 103, "015": 104, "020": 105,
        "025": 106, "030": 107, "035": 108, "040": 109,
        "045": 110, "050": 111, "055": 112, "060": 113,
        "065": 114, "070": 115, "075": 116, "080": 117,
        "085": 118, "090": 119, "095": 120, "100": 121,
        "105": 122, "110": 123, "115": 124, "120": 125
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad6998[unidad6998['subcuentas06'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['saldos06'].iloc[0]
                # Redondear el valor
                valor = round(valor, 0)
            else:
                valor = 0
                
            # Escribir en Excel
            wb['Hoja1'].cell(row=row, column=15, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=15, value=0)

# RELACIÓN A APLICAR A PRIMAS Y SINIESTROS

In [178]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad7998 existe
if 'unidad7998' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas, y si deben redondearse
    subcuentas = {
        "005": {"row": 127, "round": True},
        "010": {"row": 128, "round": True},
        "015": {"row": 129, "round": False},
        "020": {"row": 130, "round": True},
        "025": {"row": 131, "round": True},
        "030": {"row": 132, "round": False},
        "035": {"row": 133, "round": False},
        "040": {"row": 134, "round": False},
        "045": {"row": 135, "round": False},
        "050": {"row": 136, "round": False},
        "055": {"row": 137, "round": False},
        "060": {"row": 138, "round": False}
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, info in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad7998[unidad7998['subcuentas07'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['saldos07'].iloc[0]
                # Redondear el valor solo si está especificado
                if info["round"]:
                    valor = round(valor, 0)
            else:
                valor = 0
                
            # Escribir en Excel
            wb['Hoja1'].cell(row=info["row"], column=15, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=info["row"], column=15, value=0)

# RESERVA MATEMÁTICA 

In [179]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad8998 existe
if 'unidad8998' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 140,
        "010": 141,
        "015": 142,
        "020": 143,
        "025": 144
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad8998[unidad8998['subcuentas08'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['saldos08'].iloc[0]
                # Redondear el valor
                valor = round(valor, 0)
            else:
                valor = 0
                
            # Escribir en Excel
            wb['Hoja1'].cell(row=row, column=15, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=15, value=0)

# RIESGO DE SUSCRIPCION

In [180]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad9998 existe
if 'unidad9998' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 146,
        "010": 147,
        "015": 148,
        "998": 149
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad9998[unidad9998['Subcuenta'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['Saldos'].iloc[0]
                # Nota: En este caso no se redondean los valores, como en el original
            else:
                valor = 0
                
            # Escribir en Excel
            wb['Hoja1'].cell(row=row, column=15, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=15, value=0)

#  CATEGORÍA I 

In [181]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad10998 existe
if 'unidad10998' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 156, "010": 157, "015": 158, "020": 159,
        "025": 160, "030": 161, "035": 162, "040": 163,
        "045": 164, "050": 165, "055": 166, "060": 167,
        "065": 168, "999": 169
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad10998[unidad10998['Subcuenta'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['Saldos'].iloc[0]
                # Redondear el valor
                valor = round(valor, 0)
            else:
                valor = 0
                
            # Escribir en Excel
            wb['Hoja1'].cell(row=row, column=4, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=4, value=0)

# CATEGORÍA II

In [182]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad11998 existe
if 'unidad11998' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 171, "010": 172, "015": 173, "020": 174,
        "025": 175, "030": 176, "035": 177, "040": 178,
        "045": 179, "050": 180, "998": 181
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad11998[unidad11998['Subcuenta'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['Saldos'].iloc[0]
                # Redondear el valor
                valor = round(valor, 0)
            else:
                valor = 0
                
            # Escribir en Excel
            wb['Hoja1'].cell(row=row, column=4, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=4, value=0)

# CATEGORÍA II PONDERADO

In [183]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad11998_p existe
if 'unidad11998_p' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 171, "010": 172, "015": 173, "020": 174,
        "025": 175, "030": 176, "035": 177, "040": 178,
        "045": 179, "050": 180, "998": 181
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad11998_p[unidad11998_p['Subcuenta'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['Saldos'].iloc[0]
                # Redondear el valor
                valor = round(valor, 0)
            else:
                valor = 0
                
            # Escribir en Excel
            wb['Hoja1'].cell(row=row, column=7, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=7, value=0)

#  CATEGORÍA III

In [184]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad12998 existe
if 'unidad12998' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 183, "010": 184, "015": 185, "020": 186,
        "025": 187, "030": 188, "035": 189, "040": 190,
        "045": 191, "050": 192, "055": 193, "060": 194,
        "065": 195, "070": 196, "075": 197, "080": 198,
        "085": 199, "090": 200, "095": 201, "100": 202,
        "105": 203, "110": 204, "115": 205, "120": 206,
        "125": 207, "998": 208
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad12998[unidad12998['Subcuenta'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['Saldos'].iloc[0]
                # Redondear el valor
                valor = round(valor, 0)
            else:
                valor = 0
                
            # Escribir en Excel
            wb['Hoja1'].cell(row=row, column=4, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=4, value=0)

# CATEGORÍA III  DETERIORO

In [185]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad12998_d existe
if 'unidad12998_d' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 183, "010": 184, "015": 185, "020": 186,
        "025": 187, "030": 188, "035": 189, "040": 190,
        "045": 191, "050": 192, "055": 193, "060": 194,
        "065": 195, "070": 196, "075": 197, "080": 198,
        "085": 199, "090": 200, "095": 201, "100": 202,
        "105": 203, "110": 204, "115": 205, "120": 206,
        "125": 207, "998": 208
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad12998_d[unidad12998_d['Subcuenta'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['Saldos'].iloc[0]
                # Redondear el valor
                valor = round(valor, 0)
            else:
                valor = 0
                
            # Escribir en Excel
            wb['Hoja1'].cell(row=row, column=5, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=5, value=0)

# CATEGORÍA III  PONDERADO

In [186]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad12998_p existe
if 'unidad12998_p' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 183, "010": 184, "015": 185, "020": 186,
        "025": 187, "030": 188, "035": 189, "040": 190,
        "045": 191, "050": 192, "055": 193, "060": 194,
        "065": 195, "070": 196, "075": 197, "080": 198,
        "085": 199, "090": 200, "095": 201, "100": 202,
        "105": 203, "110": 204, "115": 205, "120": 206,
        "125": 207, "998": 208
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad12998_p[unidad12998_p['Subcuenta'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['Saldos'].iloc[0]
                # Redondear el valor
                valor = round(valor, 0)
            else:
                valor = 0
                
            # Escribir en Excel
            wb['Hoja1'].cell(row=row, column=7, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=7, value=0)

# RIESGO OPERACIONAL

In [187]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad13998 existe
if 'unidad13998' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 211, "010": 212, "015": 213, "020": 214, "025": 215
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad13998[unidad13998['Subcuenta'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['Saldos'].iloc[0]
                # Redondear el valor
                valor = round(valor, 0)
            else:
                valor = 0
                
            # Escribir en Excel
            wb['Hoja1'].cell(row=row, column=4, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=4, value=0)

# PATRIMONIO ADECUADO

In [188]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad14998 existe
if 'unidad14998' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 218, "010": 219, "015": 220, "020": 221,
        "025": 222, "030": 223, "035": 224, "040": 225,
        "998": 226
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad14998[unidad14998['Subcuenta'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['Saldos'].iloc[0]
                # Redondear el valor
                valor = round(valor, 0)
            else:
                valor = 0
                
            # Escribir en Excel
            wb['Hoja1'].cell(row=row, column=4, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=4, value=0)

# PATRIMONIO TÉCNICO

In [189]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad15998 existe
if 'unidad15998' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 229, "010": 230, "015": 231, "020": 232, "998": 233
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad15998[unidad15998['Subcuenta'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['Saldos'].iloc[0]
                # Redondear el valor
                valor = round(valor, 0)
            else:
                valor = 0
                
            # Escribir en Excel
            wb['Hoja1'].cell(row=row, column=4, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=4, value=0)

# LIQUIDACIÓN DE PATRIMONIO MÍNIMO EN FUNCIÓN AL ADECUADO

In [190]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad16998 existe
if 'unidad16998' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 236, "010": 237, "015": 238
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad16998[unidad16998['Subcuenta'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['Saldos'].iloc[0]
                # Redondear el valor
                valor = round(valor, 0)
            else:
                valor = 0
                
            # Escribir en Excel
            wb['Hoja1'].cell(row=row, column=4, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=4, value=0)

# ACTIVOS INTANGIBLES

In [191]:
# Filtrar los subconjuntos
AÑO_ACTUALSubset_2 = AÑO_ACTUAL[AÑO_ACTUAL['Cuenta alternativa'].astype(str).str.contains("191135", na=False)]
AÑO_ACTUALSubset_3 = AÑO_ACTUAL[AÑO_ACTUAL['Cuenta alternativa'].astype(str).str.contains("191165", na=False)]
AÑO_ACTUALSubset = pd.concat([AÑO_ACTUALSubset_2, AÑO_ACTUALSubset_3])

# Crear DataFrame Activos_Intangibles
Activos_Intangibles = AÑO_ACTUALSubset.copy()
Activos_Intangibles = Activos_Intangibles[['Cuenta alternativa', 'Total Cuenta', 'Saldo Periodo 15', 'Ejercicio']]
Activos_Intangibles.columns = ['Cuenta alternativa', 'Saldos', 'saldos_ant', 'Ejercicio']

# Procesar datos
Activos_Intangibles['Cuenta'] = Activos_Intangibles['Cuenta alternativa'].astype(str).str[:6].astype(float)
Activos_Intangibles = pd.merge(Activos_Intangibles, CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
Activos_Intangibles = Activos_Intangibles.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas adicionales
Activos_Intangibles['Unidad de captura'] = 6
Activos_Intangibles['Subcuenta'] = "055"
Activos_Intangibles['Diferencia mes a mes'] = Activos_Intangibles['Saldos'] - Activos_Intangibles['saldos_ant']
Activos_Intangibles['Diferencia %'] = (Activos_Intangibles['Saldos'] - Activos_Intangibles['saldos_ant']) / Activos_Intangibles['Saldos']
Activos_Intangibles = Activos_Intangibles.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Seleccionar y reordenar columnas
columns = ['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 'Nombre Cuenta',
           'Saldos', 'Saldos del mes pasado', 'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']
Activos_Intangibles = Activos_Intangibles[columns]

# Agregar a bd_analisis_cuentas si hay datos
if len(Activos_Intangibles) > 0:
    Activos_Intangibles['notas'] = ""
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, Activos_Intangibles])

# Calcular total absoluto
total = abs(Activos_Intangibles['Saldos'].sum())
Activos_Intangibles = pd.DataFrame({'total': [total]})

# Escribir en Excel
if len(Activos_Intangibles) == 0:
    Activos_Intangibles = pd.DataFrame({'Total': [0]})

wb['Hoja1'].cell(row=57, column=9, value=Activos_Intangibles.iloc[0,0])

<Cell 'Hoja1'.I57>

# IMPRIMIR PROFORMA

In [192]:
wb.save("MS_COGN_F407.xlsx")

# PROFORMA EN TXT 

In [192]:
# Formato sin notación científica pero sin imprimir
numero = 1e11
numero_formateado = f"{numero:.0f}"  # Convierte a formato no científico (100000000000)

# Desactivar notación científica en numpy (opcional)
np.set_printoptions(suppress=True)

# Lista vacía equivalente a bd=c() en R
bd = []

In [193]:
# Crear una lista vacía
bd = []

# Añadir segunda línea a la lista
segunda_linea = "000022000001100000"
bd.append(segunda_linea)  # Añadir al final

# Obtener el último día del mes anterior
from datetime import date, timedelta

hoy = date.today()
primer_dia_mes_actual = date(hoy.year, hoy.month, 1)
ultimo_dia_mes_anterior = primer_dia_mes_actual - timedelta(days=1)

# Formatear la fecha correctamente
dia = ultimo_dia_mes_anterior.strftime("%d")
mes = ultimo_dia_mes_anterior.strftime("%m")
year = ultimo_dia_mes_anterior.strftime("%Y")
fecha = dia + mes + year  # Último día del mes anterior en formato DDMMYYYY

# Obtener cantidad de registros total incluyendo el registro inicial
cantidad_registros = len(bd) + 1  # +1 porque añadiremos el registro inicial
cantidad_registros_formato = f"{cantidad_registros:04d}"  # Siempre 4 dígitos

# Crear y agregar el registro inicial usando la cantidad dinámica
registro_inicial = "00001113000001" + fecha + cantidad_registros_formato + "SGENCOLASEG0140"
bd.insert(0, registro_inicial)  # Insertar al inicio

In [194]:
bd

['00001113000001300420250002SGENCOLASEG0140', '000022000001100000']

#  CARGUE DEL FORMATO 407 POR UNIDAD DE CAPTURA

#  UNIDAD DE CAPTURA 1

In [195]:
# Leer el archivo Excel directamente con pandas
formato407_unidad01_COGN = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/GENERALES/Bases/Proforma/MS_COGN_F407.xlsx",
    sheet_name=0,  # Primera hoja
    skiprows=22,   # Saltar las primeras 22 filas
    nrows=23,      # Leer 23 filas (B23:D45)
    usecols="B:D", # Usar columnas B a D
    header=None    # No usar primera fila como encabezado
)
formato407_unidad01_COGN.columns = ["SUBCUENTA", "CONCEPTO", "Saldo"]

# Criterios
datos = formato407_unidad01_COGN  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 1  # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

cod_columna = 3

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)


# UNIDAD DE CAPTURA 2

In [196]:
# Leer el archivo Excel directamente con pandas
formato407_unidad02_COGN = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/GENERALES/Bases/Proforma/MS_COGN_F407.xlsx",
    sheet_name=0,  # Primera hoja
    skiprows=47,   # Saltar las primeras 47 filas
    nrows=16,      # Leer 16 filas (B48:D63)
    usecols="B:D", # Usar columnas B a D
    header=None    # No usar primera fila como encabezado
)
formato407_unidad02_COGN.columns = ["SUBCUENTA", "CONCEPTO", "Saldo"]

# Criterios
datos = formato407_unidad02_COGN  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 2  # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 3
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)


#  UNIDAD DE CAPTURA 3 

In [197]:
# Leer el archivo Excel directamente con pandas (con la ruta actualizada)
formato407_unidad03_COGN = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/GENERALES/Bases/Proforma/MS_COGN_F407.xlsx",
    sheet_name=0,  # Primera hoja
    skiprows=65,   # Saltar las primeras 65 filas
    nrows=11,      # Leer 11 filas (B66:D76)
    usecols="B:D", # Usar columnas B a D
    header=None    # No usar primera fila como encabezado
)
formato407_unidad03_COGN.columns = ["SUBCUENTA", "CONCEPTO", "Saldo"]

# Criterios
datos = formato407_unidad03_COGN  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 3  # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 3
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)


# UNIDAD DE CAPTURA 4 

In [198]:
# Leer el archivo Excel directamente con pandas (con la ruta actualizada)
formato407_unidad04_COGN = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/GENERALES/Bases/Proforma/MS_COGN_F407.xlsx",
    sheet_name=0,  # Primera hoja
    skiprows=78,   # Saltar las primeras 78 filas
    nrows=7,       # Leer 7 filas (B79:D85)
    usecols="B:D", # Usar columnas B a D
    header=None    # No usar primera fila como encabezado
)
formato407_unidad04_COGN.columns = ["SUBCUENTA", "CONCEPTO", "Saldo"]

# Criterios
datos = formato407_unidad04_COGN  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 4  # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 3
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 10 

In [199]:
# Leer el archivo Excel (usando pandas en lugar de read_excel de R)
wb = load_workbook("D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/GENERALES/Bases/Proforma/MS_COGN_F407.xlsx")
ws = wb.active
formato407_unidad10_COGN = []

# Leer rango B156:D169
for row in ws["B156:D169"]:
    row_data = []
    for cell in row:
        row_data.append(cell.value if cell.value is not None else "")
    formato407_unidad10_COGN.append(row_data)

# Convertir a DataFrame y renombrar columnas
formato407_unidad10_COGN = pd.DataFrame(formato407_unidad10_COGN)
formato407_unidad10_COGN.columns = ["SUBCUENTA", "CONCEPTO", "Saldo"]

# Criterios
datos = formato407_unidad10_COGN  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 10  # Colocar la unidad de captura usada
cod_columna = 3

# Asumimos que bd es una variable global que ya existe
# Si no existe, la inicializamos como una lista vacía
try:
    bd
except NameError:
    bd = []

secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR
limpieza01 = datos.iloc[:, cod_columna-1] != 0  # Ajuste por indexación 0-based en Python
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    d3 = d1.iloc[:, cod_columna-1].values.tolist()
    
    # Encontrar valores que cumplen la condición y redondearlos
    for i in range(len(d3)):
        if 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    valor = [str(v).zfill(20) for v in d3]
    signo = ['+' if v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz (equivalente a cbind en R)
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Concatenar todas las columnas (equivalente a paste0 en R)
    resul02 = []
    for i in range(len(matriz01)):
        row_str = (matriz01['num_secuencia'][i] + matriz01['tipo_registro'][i] + 
                  matriz01['cod_formato'][i] + matriz01['col'][i] + 
                  matriz01['unidad'][i] + matriz01['subcuenta'][i] + 
                  matriz01['signo'][i] + matriz01['valor'][i])
        resul02.append(row_str)
    
    # Agregar a la lista bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 11 

In [200]:
# Leer el archivo Excel
formato407_unidad11_COGN = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/GENERALES/Bases/Proforma/MS_COGN_F407.xlsx",
    sheet_name=0,  # Primera hoja
    skiprows=170,  # Saltar las primeras 170 filas (para llegar a B171)
    nrows=11,      # Leer 11 filas (B171:G181)
    usecols="B:G", # Usar columnas B a G
    header=None    # No usar primera fila como encabezado
)
formato407_unidad11_COGN.columns = ["SUBCUENTA", "CONCEPTO", "Saldo", "Col4", "Col5", "Col6"] 
# Agregué nombres para las columnas D-G que estaban en el rango original pero no se usaban

# Criterios
datos = formato407_unidad11_COGN  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 11  # Colocar la unidad de captura usada
cod_columna = 3

# Asumimos que bd es una variable global que ya existe
# Si no existe, la inicializamos como una lista vacía
try:
    bd
except NameError:
    bd = []

secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0  # Ajuste por indexación 0-based en Python
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02 (concatenar todas las columnas)
    resul02 = []
    for i in range(len(matriz01)):
        row_str = (matriz01['num_secuencia'][i] + 
                  matriz01['tipo_registro'][i] + 
                  matriz01['cod_formato'][i] + 
                  matriz01['col'][i] + 
                  matriz01['unidad'][i] + 
                  matriz01['subcuenta'][i] + 
                  matriz01['signo'][i] + 
                  matriz01['valor'][i])
        resul02.append(row_str)
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 12

In [201]:
# Leer el archivo Excel
formato407_unidad12_COGN = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/GENERALES/Bases/Proforma/MS_COGN_F407.xlsx",
    sheet_name=0,  # Primera hoja
    skiprows=182,  # Saltar las primeras 182 filas (para llegar a B183)
    nrows=26,      # Leer 26 filas (B183:G208)
    usecols="B:G", # Usar columnas B a G
    header=None    # No usar primera fila como encabezado
)
formato407_unidad12_COGN.columns = ["SUBCUENTA", "CONCEPTO", "Saldo", "Col4", "Col5", "Col6"]
# Añadidos nombres para las columnas adicionales que están en el rango pero no se utilizan

# Criterios
datos = formato407_unidad12_COGN  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 12  # Colocar la unidad de captura usada
cod_columna = 3

# Asumimos que bd es una variable global que ya existe
# Si no existe, la inicializamos como una lista vacía
try:
    bd
except NameError:
    bd = []

secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0  # Ajuste por indexación 0-based en Python
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02 (concatenar todas las columnas)
    resul02 = []
    for i in range(len(matriz01)):
        row_str = (matriz01['num_secuencia'][i] + 
                  matriz01['tipo_registro'][i] + 
                  matriz01['cod_formato'][i] + 
                  matriz01['col'][i] + 
                  matriz01['unidad'][i] + 
                  matriz01['subcuenta'][i] + 
                  matriz01['signo'][i] + 
                  matriz01['valor'][i])
        resul02.append(row_str)
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 13

In [202]:
# Leer el archivo Excel
formato407_unidad13_COGN = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/GENERALES/Bases/Proforma/MS_COGN_F407.xlsx",
    sheet_name=0,  # Primera hoja
    skiprows=210,  # Saltar las primeras 210 filas (para llegar a B211)
    nrows=5,       # Leer 5 filas (B211:D215)
    usecols="B:D", # Usar columnas B a D
    header=None    # No usar primera fila como encabezado
)
formato407_unidad13_COGN.columns = ["SUBCUENTA", "CONCEPTO", "Saldo"]

# Criterios
datos = formato407_unidad13_COGN  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 13  # Colocar la unidad de captura usada
cod_columna = 3

# Asumimos que bd es una variable global que ya existe
# Si no existe, la inicializamos como una lista vacía
try:
    bd
except NameError:
    bd = []

secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0  # Ajuste por indexación 0-based en Python
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02 (concatenar todas las columnas)
    resul02 = []
    for i in range(len(matriz01)):
        row_str = (matriz01['num_secuencia'][i] + 
                  matriz01['tipo_registro'][i] + 
                  matriz01['cod_formato'][i] + 
                  matriz01['col'][i] + 
                  matriz01['unidad'][i] + 
                  matriz01['subcuenta'][i] + 
                  matriz01['signo'][i] + 
                  matriz01['valor'][i])
        resul02.append(row_str)
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 14 

In [203]:
# Leer el archivo Excel
formato407_unidad14_COGN = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/GENERALES/Bases/Proforma/MS_COGN_F407.xlsx",
    sheet_name=0,  # Primera hoja
    skiprows=217,  # Saltar las primeras 217 filas (para llegar a B218)
    nrows=9,       # Leer 9 filas (B218:D226)
    usecols="B:D", # Usar columnas B a D
    header=None    # No usar primera fila como encabezado
)
formato407_unidad14_COGN.columns = ["SUBCUENTA", "CONCEPTO", "Saldo"]

# Criterios
datos = formato407_unidad14_COGN  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 14  # Colocar la unidad de captura usada
cod_columna = 3

# Asumimos que bd es una variable global que ya existe
# Si no existe, la inicializamos como una lista vacía
try:
    bd
except NameError:
    bd = []

secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0  # Ajuste por indexación 0-based en Python
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # La siguiente línea está comentada en el código original
    # for i in range(len(d3)):
    #     if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
    #         d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02 (concatenar todas las columnas)
    resul02 = []
    for i in range(len(matriz01)):
        row_str = (matriz01['num_secuencia'][i] + 
                  matriz01['tipo_registro'][i] + 
                  matriz01['cod_formato'][i] + 
                  matriz01['col'][i] + 
                  matriz01['unidad'][i] + 
                  matriz01['subcuenta'][i] + 
                  matriz01['signo'][i] + 
                  matriz01['valor'][i])
        resul02.append(row_str)
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 15

In [204]:
# Leer el archivo Excel
formato407_unidad15_COGN = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/GENERALES/Bases/Proforma/MS_COGN_F407.xlsx",
    sheet_name=0,  # Primera hoja
    skiprows=228,  # Saltar las primeras 228 filas (para llegar a B229)
    nrows=5,       # Leer 5 filas (B229:D233)
    usecols="B:D", # Usar columnas B a D
    header=None    # No usar primera fila como encabezado
)
formato407_unidad15_COGN.columns = ["SUBCUENTA", "CONCEPTO", "Saldo"]

# Criterios
datos = formato407_unidad15_COGN  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 15  # Colocar la unidad de captura usada
cod_columna = 3

# Asumimos que bd es una variable global que ya existe
# Si no existe, la inicializamos como una lista vacía
try:
    bd
except NameError:
    bd = []

secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0  # Ajuste por indexación 0-based en Python
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02 (concatenar todas las columnas)
    resul02 = []
    for i in range(len(matriz01)):
        row_str = (matriz01['num_secuencia'][i] + 
                  matriz01['tipo_registro'][i] + 
                  matriz01['cod_formato'][i] + 
                  matriz01['col'][i] + 
                  matriz01['unidad'][i] + 
                  matriz01['subcuenta'][i] + 
                  matriz01['signo'][i] + 
                  matriz01['valor'][i])
        resul02.append(row_str)
    
    # Añadir a bd
    bd.extend(resul02)

#  UNIDAD DE CAPTURA 16

In [205]:
# Leer el archivo Excel
formato407_unidad16_COGN = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/GENERALES/Bases/Proforma/MS_COGN_F407.xlsx",
    sheet_name=0,  # Primera hoja
    skiprows=235,  # Saltar las primeras 235 filas (para llegar a B236)
    nrows=3,       # Leer 3 filas (B236:D238)
    usecols="B:D", # Usar columnas B a D
    header=None    # No usar primera fila como encabezado
)
formato407_unidad16_COGN.columns = ["SUBCUENTA", "CONCEPTO", "Saldo"]

# Criterios
datos = formato407_unidad16_COGN  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 16  # Colocar la unidad de captura usada
cod_columna = 3

# Asumimos que bd es una variable global que ya existe
# Si no existe, la inicializamos como una lista vacía
try:
    bd
except NameError:
    bd = []

secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0  # Ajuste por indexación 0-based en Python
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02 (concatenar todas las columnas)
    resul02 = []
    for i in range(len(matriz01)):
        row_str = (matriz01['num_secuencia'][i] + 
                  matriz01['tipo_registro'][i] + 
                  matriz01['cod_formato'][i] + 
                  matriz01['col'][i] + 
                  matriz01['unidad'][i] + 
                  matriz01['subcuenta'][i] + 
                  matriz01['signo'][i] + 
                  matriz01['valor'][i])
        resul02.append(row_str)
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 12

In [206]:
# Leer el archivo Excel
formato407_unidad12_COGN = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/GENERALES/Bases/Proforma/MS_COGN_F407.xlsx",
    sheet_name=0,  # Primera hoja
    skiprows=182,  # Saltar las primeras 182 filas (para llegar a B183)
    nrows=26,      # Leer 26 filas (B183:G208)
    usecols="B:G", # Usar columnas B a G
    header=None    # No usar primera fila como encabezado
)
formato407_unidad12_COGN.columns = ["SUBCUENTA", "CONCEPTO", "Saldo", "Col4", "Col5", "Col6"]

# Criterios
datos = formato407_unidad12_COGN  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 12  # Colocar la unidad de captura usada
cod_columna = 4  # Esta es la diferencia con los ejemplos anteriores

# Asumimos que bd es una variable global que ya existe
# Si no existe, la inicializamos como una lista vacía
try:
    bd
except NameError:
    bd = []

secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0  # Ajuste por indexación 0-based en Python
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # Nótese que aquí se usa cod_columna-2
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02 (concatenar todas las columnas)
    resul02 = []
    for i in range(len(matriz01)):
        row_str = (matriz01['num_secuencia'][i] + 
                  matriz01['tipo_registro'][i] + 
                  matriz01['cod_formato'][i] + 
                  matriz01['col'][i] + 
                  matriz01['unidad'][i] + 
                  matriz01['subcuenta'][i] + 
                  matriz01['signo'][i] + 
                  matriz01['valor'][i])
        resul02.append(row_str)
    
    # Añadir a bd
    bd.extend(resul02)

#  UNIDAD DE CAPTURA 11

In [207]:
# Leer el archivo Excel
formato407_unidad11_COGN = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/GENERALES/Bases/Proforma/MS_COGN_F407.xlsx",
    sheet_name=0,  # Primera hoja
    skiprows=170,  # Saltar las primeras 170 filas (para llegar a B171)
    nrows=11,      # Leer 11 filas (B171:G181)
    usecols="B:G", # Usar columnas B a G
    header=None    # No usar primera fila como encabezado
)
formato407_unidad11_COGN.columns = ["SUBCUENTA", "CONCEPTO", "Saldo", "Col4", "Col5", "Col6"]

# Criterios
datos = formato407_unidad11_COGN  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 11  # Colocar la unidad de captura usada
cod_columna = 6  # En este caso es 6

# Asumimos que bd es una variable global que ya existe
# Si no existe, la inicializamos como una lista vacía
try:
    bd
except NameError:
    bd = []

secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0  # Ajuste por indexación 0-based en Python
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # Nótese que aquí se usa cod_columna-2
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02 (concatenar todas las columnas)
    resul02 = []
    for i in range(len(matriz01)):
        row_str = (matriz01['num_secuencia'][i] + 
                  matriz01['tipo_registro'][i] + 
                  matriz01['cod_formato'][i] + 
                  matriz01['col'][i] + 
                  matriz01['unidad'][i] + 
                  matriz01['subcuenta'][i] + 
                  matriz01['signo'][i] + 
                  matriz01['valor'][i])
        resul02.append(row_str)
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 12

In [208]:
# Leer el archivo Excel
formato407_unidad12_COGN = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/GENERALES/Bases/Proforma/MS_COGN_F407.xlsx",
    sheet_name=0,  # Primera hoja
    skiprows=182,  # Saltar las primeras 182 filas (para llegar a B183)
    nrows=26,      # Leer 26 filas (B183:G208)
    usecols="B:G", # Usar columnas B a G
    header=None    # No usar primera fila como encabezado
)
formato407_unidad12_COGN.columns = ["SUBCUENTA", "CONCEPTO", "Saldo", "Col4", "Col5", "Col6"]

# Criterios
datos = formato407_unidad12_COGN  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 12  # Colocar la unidad de captura usada
cod_columna = 6  # En este caso es 6, no 4 como en la versión anterior

# Asumimos que bd es una variable global que ya existe
# Si no existe, la inicializamos como una lista vacía
try:
    bd
except NameError:
    bd = []

secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0  # Ajuste por indexación 0-based en Python
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # Nótese que aquí se usa cod_columna-2 (que es 4)
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02 (concatenar todas las columnas)
    resul02 = []
    for i in range(len(matriz01)):
        row_str = (matriz01['num_secuencia'][i] + 
                  matriz01['tipo_registro'][i] + 
                  matriz01['cod_formato'][i] + 
                  matriz01['col'][i] + 
                  matriz01['unidad'][i] + 
                  matriz01['subcuenta'][i] + 
                  matriz01['signo'][i] + 
                  matriz01['valor'][i])
        resul02.append(row_str)
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 5 

In [209]:
# Leer el archivo Excel
formato407_unidad05_COGN = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/GENERALES/Bases/Proforma/MS_COGN_F407.xlsx",
    sheet_name=0,  # Primera hoja
    skiprows=92,   # Saltar las primeras 92 filas (para llegar a B93)
    nrows=8,       # Leer 8 filas (B93:O100)
    usecols="B:O", # Usar columnas B a O
    header=None    # No usar primera fila como encabezado
)

# Asignar nombres a las columnas
formato407_unidad05_COGN.columns = [
    "SUBCUENTA", "CONCEPTO", "Saldo", "Deterioro", "Exposición  Crediticia", 
    "Valor Ponderado", "Educativo", "Vida Individual", 
    "Pensiones con Conmutación Pensional", "Pensiones Ley 100",
    "Pensiones Voluntarias", "Riesgos Profesionales", 
    "Previsionales de Invalidez y Sobrevivencia", "Demas_ramos"
]

# Criterios
datos = formato407_unidad05_COGN  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 5  # Colocar la unidad de captura usada
cod_columna = 14  # En este caso es 14

# Asumimos que bd es una variable global que ya existe
# Si no existe, la inicializamos como una lista vacía
try:
    bd
except NameError:
    bd = []

secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0  # Ajuste por indexación 0-based en Python
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # cod_columna-2 (14-2=12)
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02 (concatenar todas las columnas)
    resul02 = []
    for i in range(len(matriz01)):
        row_str = (matriz01['num_secuencia'][i] + 
                  matriz01['tipo_registro'][i] + 
                  matriz01['cod_formato'][i] + 
                  matriz01['col'][i] + 
                  matriz01['unidad'][i] + 
                  matriz01['subcuenta'][i] + 
                  matriz01['signo'][i] + 
                  matriz01['valor'][i])
        resul02.append(row_str)
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 6

In [210]:
# Leer el archivo Excel
formato407_unidad06_COGN = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/GENERALES/Bases/Proforma/MS_COGN_F407.xlsx",
    sheet_name=0,  # Primera hoja
    skiprows=101,  # Saltar las primeras 101 filas (para llegar a B102)
    nrows=24,      # Leer 24 filas (B102:O125)
    usecols="B:O", # Usar columnas B a O
    header=None    # No usar primera fila como encabezado
)

# Asignar nombres a las columnas
formato407_unidad06_COGN.columns = [
    "SUBCUENTA", "CONCEPTO", "Saldo", "Deterioro", "Exposición  Crediticia", 
    "Valor Ponderado", "Educativo", "Vida Individual", 
    "Pensiones con Conmutación Pensional", "Pensiones Ley 100",
    "Pensiones Voluntarias", "Riesgos Profesionales", 
    "Previsionales de Invalidez y Sobrevivencia", "Demas_ramos"
]

# Criterios
datos = formato407_unidad06_COGN  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 6  # Colocar la unidad de captura usada
cod_columna = 14  # En este caso es 14

# Asumimos que bd es una variable global que ya existe
# Si no existe, la inicializamos como una lista vacía
try:
    bd
except NameError:
    bd = []

secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0  # Ajuste por indexación 0-based en Python
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # cod_columna-2 (14-2=12)
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02 (concatenar todas las columnas)
    resul02 = []
    for i in range(len(matriz01)):
        row_str = (matriz01['num_secuencia'][i] + 
                  matriz01['tipo_registro'][i] + 
                  matriz01['cod_formato'][i] + 
                  matriz01['col'][i] + 
                  matriz01['unidad'][i] + 
                  matriz01['subcuenta'][i] + 
                  matriz01['signo'][i] + 
                  matriz01['valor'][i])
        resul02.append(row_str)
    
    # Añadir a bd
    bd.extend(resul02)

#  UNIDAD DE CAPTURA 7 

In [211]:
# Leer el archivo Excel
formato407_unidad07_COGN = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/GENERALES/Bases/Proforma/MS_COGN_F407.xlsx",
    sheet_name=0,  # Primera hoja
    skiprows=126,  # Saltar las primeras 126 filas (para llegar a B127)
    nrows=12,      # Leer 12 filas (B127:O138)
    usecols="B:O", # Usar columnas B a O
    header=None    # No usar primera fila como encabezado
)

# Asignar nombres a las columnas
formato407_unidad07_COGN.columns = [
    "SUBCUENTA", "CONCEPTO", "Saldo", "Deterioro", "Exposición  Crediticia", 
    "Valor Ponderado", "Educativo", "Vida Individual", 
    "Pensiones con Conmutación Pensional", "Pensiones Ley 100",
    "Pensiones Voluntarias", "Riesgos Profesionales", 
    "Previsionales de Invalidez y Sobrevivencia", "Demas_ramos"
]

# Criterios
datos = formato407_unidad07_COGN  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 7  # Colocar la unidad de captura usada
cod_columna = 14  # En este caso es 14

# Asumimos que bd es una variable global que ya existe
# Si no existe, la inicializamos como una lista vacía
try:
    bd
except NameError:
    bd = []

secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0  # Ajuste por indexación 0-based en Python
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # cod_columna-2 (14-2=12)
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02 (concatenar todas las columnas)
    resul02 = []
    for i in range(len(matriz01)):
        row_str = (matriz01['num_secuencia'][i] + 
                  matriz01['tipo_registro'][i] + 
                  matriz01['cod_formato'][i] + 
                  matriz01['col'][i] + 
                  matriz01['unidad'][i] + 
                  matriz01['subcuenta'][i] + 
                  matriz01['signo'][i] + 
                  matriz01['valor'][i])
        resul02.append(row_str)
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 8

In [212]:
# Leer el archivo Excel
formato407_unidad08_COGN = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/GENERALES/Bases/Proforma/MS_COGN_F407.xlsx",
    sheet_name=0,  # Primera hoja
    skiprows=139,  # Saltar las primeras 139 filas (para llegar a B140)
    nrows=5,       # Leer 5 filas (B140:O144)
    usecols="B:O", # Usar columnas B a O
    header=None    # No usar primera fila como encabezado
)

# Asignar nombres a las columnas
formato407_unidad08_COGN.columns = [
    "SUBCUENTA", "CONCEPTO", "Saldo", "Deterioro", "Exposición  Crediticia", 
    "Valor Ponderado", "Educativo", "Vida Individual", 
    "Pensiones con Conmutación Pensional", "Pensiones Ley 100",
    "Pensiones Voluntarias", "Riesgos Profesionales", 
    "Previsionales de Invalidez y Sobrevivencia", "Demas_ramos"
]

# Criterios
datos = formato407_unidad08_COGN  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 8  # Colocar la unidad de captura usada
cod_columna = 14  # En este caso es 14

# Asumimos que bd es una variable global que ya existe
# Si no existe, la inicializamos como una lista vacía
try:
    bd
except NameError:
    bd = []

secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0  # Ajuste por indexación 0-based en Python
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # cod_columna-2 (14-2=12)
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02 (concatenar todas las columnas)
    resul02 = []
    for i in range(len(matriz01)):
        row_str = (matriz01['num_secuencia'][i] + 
                  matriz01['tipo_registro'][i] + 
                  matriz01['cod_formato'][i] + 
                  matriz01['col'][i] + 
                  matriz01['unidad'][i] + 
                  matriz01['subcuenta'][i] + 
                  matriz01['signo'][i] + 
                  matriz01['valor'][i])
        resul02.append(row_str)
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 9 

In [213]:
# Leer el archivo Excel
formato407_unidad09_COGN = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/GENERALES/Bases/Proforma/MS_COGN_F407.xlsx",
    sheet_name=0,  # Primera hoja
    skiprows=145,  # Saltar las primeras 145 filas (para llegar a B146)
    nrows=4,       # Leer 4 filas (B146:O149)
    usecols="B:O", # Usar columnas B a O
    header=None    # No usar primera fila como encabezado
)

# Asignar nombres a las columnas
formato407_unidad09_COGN.columns = [
    "SUBCUENTA", "CONCEPTO", "Saldo", "Deterioro", "Exposición  Crediticia", 
    "Valor Ponderado", "Educativo", "Vida Individual", 
    "Pensiones con Conmutación Pensional", "Pensiones Ley 100",
    "Pensiones Voluntarias", "Riesgos Profesionales", 
    "Previsionales de Invalidez y Sobrevivencia", "Demas_ramos"
]

# Criterios
datos = formato407_unidad09_COGN  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 9  # Colocar la unidad de captura usada
cod_columna = 14  # En este caso es 14

# Asumimos que bd es una variable global que ya existe
# Si no existe, la inicializamos como una lista vacía
try:
    bd
except NameError:
    bd = []

secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0  # Ajuste por indexación 0-based en Python
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # cod_columna-2 (14-2=12)
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02 (concatenar todas las columnas)
    resul02 = []
    for i in range(len(matriz01)):
        row_str = (matriz01['num_secuencia'][i] + 
                  matriz01['tipo_registro'][i] + 
                  matriz01['cod_formato'][i] + 
                  matriz01['col'][i] + 
                  matriz01['unidad'][i] + 
                  matriz01['subcuenta'][i] + 
                  matriz01['signo'][i] + 
                  matriz01['valor'][i])
        resul02.append(row_str)
    
    # Añadir a bd
    bd.extend(resul02)

# CARGUE DEL FORMATO 481 POR UNIDAD DE CAPTURA

# UNIDAD DE CAPTURA 1

In [214]:
# Leer el archivo Excel
# Opción 1: Usando pandas
formato481_unidad01_COGN = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/GENERALES/Bases/Fuentes de Información COGN.xlsx",
    sheet_name="Formato481",
    skiprows=28,  # Ajustado para empezar en la fila 29
    nrows=25,     # Leer 25 filas (b29:ad53)
    usecols="B:AD",  # Usar columnas B a AD
    header=None   # No usar primera fila como encabezado
)

# Asignar nombres a las columnas
formato481_unidad01_COGN.columns = [
    "SUBCUENTA", "CONCEPTO", "ELIMINADA", "RESERVA MATEMÁTICA", "DEPÓSITO DE RESERVA A REASEGURADORES",
    "RESERVA DESVIACIÓN DE SINIESTRALIDAD", "RESERVA PARA SINIESTROS AVISADOS", "RESERVA PARA SINIESTROS NO AVISADOS",
    "RESERVA PARA SINIESTROS PENDIENTES GARANTIZADOS POR LA NACIÓN", "RESERVA ENFERMEDAD LABORAL",
    "TOTAL RESERVAS", "RESERVAS TECNICAS EN MONEDA NACIONAL", "RESERVAS TECNICAS EN MONEDA EXTRANJERA",
    "TOTAL ACTIVOS QUE RESPALDAN  RESERVAS TÉCNICAS", "INVERSIONES MONEDA NACIONAL QUE RESPALDAN LAS RESERVAS TECNICAS",
    "INVERSIONES MONEDA EXTRANJERA QUE RESPALDAN LAS RESERVAS TECNICAS", "TOTAL OTROS ACTIVOS QUE RESPALDAN LA RESERVAS TECNICAS",
    "DEFECTO", "RESERVA DE INSUFICIENCIA DE ACTIVOS", "RESERVA DE RIESGOS CATASTRÓFICOS", "ELIMINADA", "RESERVA DE PRIMA NO DEVENGADA",
    "RESERVA POR INSUFICIENCIA DE PRIMA", "CONTINGENCIAS A CARGO DE REASEGURADORES EN MONEDA NACIONAL",
    "CONTINGENCIAS A CARGO DE REASEGURADORES EN MONEDA EXTRANJERA", "TOTAL CONTINGENCIAS A CARGO DE REASEGURADORES",
    "TOTAL INVERSIONES QUE RESPALDAN  LAS RESERVAS TÉCNICAS", "OTROS ACTIVOS EN MONEDA NACIONAL  QUE RESPALDAN LAS RESERVAS TÉCNICAS",
    "OTROS ACTIVOS EN MONEDA EXTRANJERA QUE RESPALDAN LAS RESERVAS TÉCNICAS"
]

# Criterios
datos = formato481_unidad01_COGN  # Colocar la bd usada
cod_formato = 481  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 1  # Colocar la unidad de captura usada 

# Inicializar bd si no existe
try:
    bd
except NameError:
    bd = []

# Recorrer las columnas (equivalente a for (cod_columna in 1:(ncol(datos)-2)))
for cod_columna in range(1, len(datos.columns) - 2 + 1):  # +1 porque range es exclusivo al final
    secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR
    
    # En R: limpieza01<-which(!datos[,cod_columna+2]==0)
    # En Python: el índice de columna es cod_columna+1 (por indexación base 0)
    limpieza01 = datos.iloc[:, cod_columna+1] != 0  # Crear máscara booleana
    limpieza01 = limpieza01 & ~datos.iloc[:, cod_columna+1].isna()  # Filtrar NaN
    d1 = datos[limpieza01].reset_index(drop=True)  # Resetear índices
    
    if len(d1) > 0:
        # Generar números de secuencia
        # En R: str_pad(seq(secuencia_numero_registro, secuencia_numero_registro+nrow(d1)), 5, pad = "0")
        num_secuencia = [f"{i:05d}" for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
        
        # En R: d3<-as.vector(as.matrix(d1[,cod_columna+2]))
        d3 = d1.iloc[:, cod_columna+1].tolist()  # Convertir a lista
        
        # En R: d3[which(d3>0 & d3<=1)]<- round(d3[which(d3>0 & d3<=1)]*100,2)
        # Multiplicar valores entre 0 y 1 por 100
        for i in range(len(d3)):
            if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
                d3[i] = round(d3[i] * 100, 2)
        
        # En R: valor=str_pad(d3,20,pad = "0")
        # Formatear valores con 20 dígitos
        valor = [f"{abs(v):020.0f}" if isinstance(v, (int, float)) and not pd.isna(v) else "00000000000000000000" for v in d3]
        
        # En R: signo=ifelse(d1[,cod_columna+2]>0,"+","-")
        # Determinar signo de valores
        signo = []
        for v in d1.iloc[:, cod_columna+1]:
            if isinstance(v, (int, float)) and not pd.isna(v):
                signo.append("+" if v > 0 else "-")
            else:
                signo.append("-")  # Default para valores no numéricos
        
        # Crear matriz01 (DataFrame en Python)
        # Formatear subcuenta con 3 dígitos
        subcuenta = []
        for v in d1["SUBCUENTA"]:
            if isinstance(v, (int, float)) and not pd.isna(v):
                subcuenta.append(f"{int(v):03d}")
            else:
                subcuenta.append("000")
        
        # Construir el DataFrame equivalente a matriz01
        matriz01 = pd.DataFrame({
            'num_secuencia': num_secuencia,
            'tipo_registro': [str(tipo_registro)] * len(d1),
            'cod_formato': [str(cod_formato)] * len(d1),
            'col': [f"{cod_columna:02d}"] * len(d1),
            'unidad': [f"{cod_unidad:02d}"] * len(d1),
            'subcuenta': subcuenta,
            'signo': signo,
            'valor': valor
        })
        
        # En R: resul02=paste0(matriz01[,1],matriz01[,2],...,matriz01[,8])
        # Concatenar las columnas para formar cada registro
        resul02 = []
        for i in range(len(matriz01)):
            registro = (
                matriz01['num_secuencia'][i] +
                matriz01['tipo_registro'][i] +
                matriz01['cod_formato'][i] +
                matriz01['col'][i] +
                matriz01['unidad'][i] +
                matriz01['subcuenta'][i] +
                matriz01['signo'][i] +
                matriz01['valor'][i]
            )
            resul02.append(registro)
        
        # En R: bd=c(bd,resul02)
        # Añadir a bd
        bd.extend(resul02)


# UNIDAD DE CAPTURA 2 

In [215]:
# Leer el archivo Excel
# Opción 1: Usando pandas
formato481_unidad02_COGN = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/GENERALES/Bases/Fuentes de Información COGN.xlsx",
    sheet_name="Formato481",
    skiprows=54,  # Ajustado para empezar en la fila 55
    nrows=2,      # Leer 2 filas (b55:ad56)
    usecols="B:AD",  # Usar columnas B a AD
    header=None   # No usar primera fila como encabezado
)

# Asignar nombres a las columnas
formato481_unidad02_COGN.columns = [
    "SUBCUENTA", "CONCEPTO", "ELIMINADA", "RESERVA MATEMÁTICA", "DEPÓSITO DE RESERVA A REASEGURADORES",
    "RESERVA DESVIACIÓN DE SINIESTRALIDAD", "RESERVA PARA SINIESTROS AVISADOS", "RESERVA PARA SINIESTROS NO AVISADOS",
    "RESERVA PARA SINIESTROS PENDIENTES GARANTIZADOS POR LA NACIÓN", "RESERVA ENFERMEDAD LABORAL",
    "TOTAL RESERVAS", "RESERVAS TECNICAS EN MONEDA NACIONAL", "RESERVAS TECNICAS EN MONEDA EXTRANJERA",
    "TOTAL ACTIVOS QUE RESPALDAN  RESERVAS TÉCNICAS", "INVERSIONES MONEDA NACIONAL QUE RESPALDAN LAS RESERVAS TECNICAS",
    "INVERSIONES MONEDA EXTRANJERA QUE RESPALDAN LAS RESERVAS TECNICAS", "TOTAL OTROS ACTIVOS QUE RESPALDAN LA RESERVAS TECNICAS",
    "DEFECTO", "RESERVA DE INSUFICIENCIA DE ACTIVOS", "RESERVA DE RIESGOS CATASTRÓFICOS", "ELIMINADA", "RESERVA DE PRIMA NO DEVENGADA",
    "RESERVA POR INSUFICIENCIA DE PRIMA", "CONTINGENCIAS A CARGO DE REASEGURADORES EN MONEDA NACIONAL",
    "CONTINGENCIAS A CARGO DE REASEGURADORES EN MONEDA EXTRANJERA", "TOTAL CONTINGENCIAS A CARGO DE REASEGURADORES",
    "TOTAL INVERSIONES QUE RESPALDAN  LAS RESERVAS TÉCNICAS", "OTROS ACTIVOS EN MONEDA NACIONAL  QUE RESPALDAN LAS RESERVAS TÉCNICAS",
    "OTROS ACTIVOS EN MONEDA EXTRANJERA QUE RESPALDAN LAS RESERVAS TÉCNICAS"
]

# Criterios
datos = formato481_unidad02_COGN  # Colocar la bd usada
cod_formato = 481  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 2  # Colocar la unidad de captura usada

# Inicializar bd si no existe
try:
    bd
except NameError:
    bd = []

# Recorrer las columnas (equivalente a for (cod_columna in 1:(ncol(datos)-2)))
for cod_columna in range(1, len(datos.columns) - 2 + 1):  # +1 porque range es exclusivo al final
    secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR
    
    # En R: limpieza01<-which(!datos[,cod_columna+2]==0)
    # En Python: el índice de columna es cod_columna+1 (por indexación base 0)
    limpieza01 = datos.iloc[:, cod_columna+1] != 0  # Crear máscara booleana
    limpieza01 = limpieza01 & ~datos.iloc[:, cod_columna+1].isna()  # Filtrar NaN
    d1 = datos[limpieza01].reset_index(drop=True)  # Resetear índices
    
    if len(d1) > 0:
        # Generar números de secuencia
        # En R: str_pad(seq(secuencia_numero_registro, secuencia_numero_registro+nrow(d1)), 5, pad = "0")
        num_secuencia = [f"{i:05d}" for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
        
        # En R: d3<-as.vector(as.matrix(d1[,cod_columna+2]))
        d3 = d1.iloc[:, cod_columna+1].tolist()  # Convertir a lista
        
        # En R: d3[which(d3>0 & d3<=1)]<- round(d3[which(d3>0 & d3<=1)]*100,2)
        # Multiplicar valores entre 0 y 1 por 100
        for i in range(len(d3)):
            if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
                d3[i] = round(d3[i] * 100, 2)
        
        # En R: valor=str_pad(d3,20,pad = "0")
        # Formatear valores con 20 dígitos
        valor = [f"{abs(v):020.0f}" if isinstance(v, (int, float)) and not pd.isna(v) else "00000000000000000000" for v in d3]
        
        # En R: signo=ifelse(d1[,cod_columna+2]>0,"+","-")
        # Determinar signo de valores
        signo = []
        for v in d1.iloc[:, cod_columna+1]:
            if isinstance(v, (int, float)) and not pd.isna(v):
                signo.append("+" if v > 0 else "-")
            else:
                signo.append("-")  # Default para valores no numéricos
        
        # Crear matriz01 (DataFrame en Python)
        # Formatear subcuenta con 3 dígitos
        subcuenta = []
        for v in d1["SUBCUENTA"]:
            if isinstance(v, (int, float)) and not pd.isna(v):
                subcuenta.append(f"{int(v):03d}")
            else:
                subcuenta.append("000")
        
        # Construir el DataFrame equivalente a matriz01
        matriz01 = pd.DataFrame({
            'num_secuencia': num_secuencia,
            'tipo_registro': [str(tipo_registro)] * len(d1),
            'cod_formato': [str(cod_formato)] * len(d1),
            'col': [f"{cod_columna:02d}"] * len(d1),
            'unidad': [f"{cod_unidad:02d}"] * len(d1),
            'subcuenta': subcuenta,
            'signo': signo,
            'valor': valor
        })
        
        # En R: resul02=paste0(matriz01[,1],matriz01[,2],...,matriz01[,8])
        # Concatenar las columnas para formar cada registro
        resul02 = []
        for i in range(len(matriz01)):
            registro = (
                matriz01['num_secuencia'][i] +
                matriz01['tipo_registro'][i] +
                matriz01['cod_formato'][i] +
                matriz01['col'][i] +
                matriz01['unidad'][i] +
                matriz01['subcuenta'][i] +
                matriz01['signo'][i] +
                matriz01['valor'][i]
            )
            resul02.append(registro)
        
        # En R: bd=c(bd,resul02)
        # Añadir a bd
        bd.extend(resul02)


#  LINEA FINAL

In [216]:
# Generar el registro final
n_final = str(len(bd) + 1).zfill(5)  # Número de registro con padding de ceros
n_final = n_final + str(tipo_registro + 1)  # Concatenar con tipo_registro+1
bd.append(n_final)  # Añadir a la lista bd

# EXPORTAR FORMATO TXT

In [217]:
# Suponiendo que bd ya contiene todos los registros generados
# Convertir bd a DataFrame
bd1 = pd.DataFrame(bd, columns=['registro'])

# Filtrar registros que contienen "nan" o valores cero
registros_filtrados = []
encabezado = None
pie = None

# Guardar el encabezado y el pie
if len(bd) > 0:
    encabezado = bd[0]
if len(bd) > 1:
    pie = bd[-1]

# Filtrar y corregir los demás registros (excluyendo encabezado y pie)
for idx, registro in enumerate(bd):
    # Saltar el primer y último registro
    if idx == 0 or idx == len(bd) - 1:
        continue
    
    # Verificar si el registro contiene "nan"
    if "nan" not in str(registro).lower():
        # Extraer el signo y el valor
        match = re.search(r'([-+])(\d+\.\d+|\d+)$', str(registro))
        if match:
            signo = match.group(1)
            valor_str = match.group(2)
            
            try:
                # Convertir a float
                valor_float = float(valor_str)
                
                # Verificar si es cero
                if valor_float == 0:
                    continue  # Saltar este registro
                
                # Determinar si es un valor porcentual (menores a 100 con decimales)
                if valor_float < 100 and '.' in valor_str:
                    # Para porcentajes, mantener el punto decimal y formatear a 2 decimales
                    valor_formateado = f"{valor_float:.2f}"
                    # Asegurar que tenga 20 caracteres incluyendo el punto
                    while len(valor_formateado) < 20:
                        valor_formateado = "0" + valor_formateado
                    nuevo_valor = valor_formateado
                else:
                    # Para valores grandes, eliminar el punto decimal
                    valor_int = int(valor_float)
                    nuevo_valor = str(valor_int).zfill(20)
                
                # Reconstruir el registro con el valor corregido
                prefijo = registro[:registro.rfind(signo) + 1]  # Todo hasta e incluyendo el signo
                nuevo_registro = prefijo + nuevo_valor
                
                registros_filtrados.append(nuevo_registro)
            except ValueError:
                # Si hay algún error en la conversión, mantener el registro original
                registros_filtrados.append(registro)
        else:
            # Si no tiene el formato esperado, mantenerlo
            registros_filtrados.append(registro)

# Renumerar los registros filtrados
registros_renumerados = []
for i, registro in enumerate(registros_filtrados):
    # Extraer las partes del registro (asumiendo formato fijo)
    nuevo_num = str(i + 2).zfill(5)  # Empezar desde 00002 porque 00001 es el encabezado
    resto_registro = registro[5:]  # Obtener todo excepto los primeros 5 caracteres
    registros_renumerados.append(nuevo_num + resto_registro)

# Reconstruir la lista completa con encabezado y pie
resultado_final = []
if encabezado:
    resultado_final.append(encabezado)
resultado_final.extend(registros_renumerados)
if pie:
    # Actualizar el número en el pie
    nuevo_pie = str(len(resultado_final) + 1).zfill(5) + pie[5:]
    resultado_final.append(nuevo_pie)

# Crear un nuevo DataFrame con los registros renumerados
bd1_final = pd.DataFrame(resultado_final, columns=['registro'])

# Paso 1: Escribir el archivo con todas las líneas excepto la última
with open("CLG.txt", "w") as f:
    for i in range(len(bd1_final) - 1):
        f.write(bd1_final.iloc[i, 0] + "\n")

# Paso 2: Añadir la última línea manualmente, sin el salto de línea final
with open("CLG.txt", "a") as f:
    f.write(bd1_final.iloc[-1, 0])

# Actualizar el contador de registros en el encabezado
if len(bd1_final) > 0:
    # Extraer el encabezado actual
    encabezado_actual = bd1_final.iloc[0, 0]
    
# Actualizar el encabezado con contador y sufijo correctos
if len(bd1_final) > 0:
    encabezado_actual = bd1_final.iloc[0, 0]

    # Tomar las partes del encabezado
    prefijo_base = encabezado_actual[:14]           # "00001113000001"
    fecha_encabezado = encabezado_actual[14:22]     # "Fecha"
    contador = str(len(bd1_final)).zfill(5)         # número exacto de registros
    sufijo = encabezado_actual[-15:]                # últimos 15 caracteres siempre son el sufijo

    # Armar nuevo encabezado
    nuevo_encabezado = prefijo_base + fecha_encabezado + contador + sufijo

    # Verificación final
    if len(nuevo_encabezado) != 42:
        raise ValueError(f"Encabezado mal formado: '{nuevo_encabezado}' tiene {len(nuevo_encabezado)} caracteres (deben ser 42)")

    # Reemplazar el encabezado
    bd1_final.iloc[0, 0] = nuevo_encabezado

    # Reescribir el archivo
    with open("CLG.txt", "w") as f:
        for i in range(len(bd1_final) - 1):
            f.write(bd1_final.iloc[i, 0] + "\n")
    with open("CLG.txt", "a") as f:
        f.write(bd1_final.iloc[-1, 0])

    
    # Reemplazar el encabezado
    bd1_final.iloc[0, 0] = nuevo_encabezado
    
    # Reescribir el archivo con el encabezado actualizado
    with open("CLG.txt", "w") as f:
        for i in range(len(bd1_final) - 1):
            f.write(bd1_final.iloc[i, 0] + "\n")
    
    # Añadir la última línea sin salto de línea
    with open("CLG.txt", "a") as f:
        f.write(bd1_final.iloc[-1, 0])

# Leer el contenido del archivo y verificar
with open("CLG.txt", "r") as f:
    lines = f.readlines()


# VALIDACIONES POR RAMOS 

In [221]:
# Lectura de los archivos de Excel
AÑO_ACTUAL = pd.read_excel(ruta1 + "Fuentes de Información COGN.xlsx", sheet_name="AÑO ACTUAL")
N1 = pd.read_excel(ruta1 + "Fuentes de Información COGN.xlsx", sheet_name="N-1")
N2 = pd.read_excel(ruta1 + "Fuentes de Información COGN.xlsx", sheet_name="N-2")
N3 = pd.read_excel(ruta1 + "Fuentes de Información COGN.xlsx", sheet_name="N-3")

# Convertir columnas a tipo numérico (valores no numéricos se convierten en NaN)
N2['Cl.movimiento'] = pd.to_numeric(N2['Cl.movimiento'], errors='coerce')
N3['Cl.movimiento'] = pd.to_numeric(N3['Cl.movimiento'], errors='coerce')

# Concatenar DataFrames (equivalente a bind_rows)
cons_unida = pd.concat([AÑO_ACTUAL, N1, N2, N3], ignore_index=True)

# Seleccionar columnas específicas y obtener valores únicos
t2 = T_homologacion_ramos.iloc[:, [1, 3]].drop_duplicates()

# Realizar joins y filtrado
cons1 = cons_unida.merge(t2, how='left')
cons1 = cons1[(cons1['Ejercicio'] == 2024) | (cons1['Ejercicio'] == 2023)]
cons1 = cons1.merge(VALIDACION_RAMOS, how='left').drop_duplicates()

# Obtener fecha actual y mes
fecha_actual = datetime.now().date()
mes_actual = fecha_actual.strftime("%B")
mes = fecha_actual.month - 1
posicion = cons1.columns.get_loc("Saldo Periodo 01")

# Filtrar y procesar datos para ejercicio 2023 o anterior
cons1a = cons1[cons1['Ejercicio'] <= 2023].copy()
cons1a['periodo_actual'] = cons1a.iloc[:, posicion+mes:27].sum(axis=1, skipna=True)
cons1a['periodo_anterior'] = cons1a.iloc[:, posicion+mes-1:27].sum(axis=1, skipna=True)
cons1a['diferencia'] = cons1a['periodo_actual'] - cons1a['periodo_anterior']

# Filtrar y procesar datos para ejercicio 2024
cons1b = cons1[cons1['Ejercicio'] == 2024].copy()
cons1b['periodo_actual'] = cons1b.iloc[:, posicion:posicion+mes].sum(axis=1, skipna=True)
cons1b['periodo_anterior'] = cons1b.iloc[:, posicion:posicion+mes-1].sum(axis=1, skipna=True)
cons1b['diferencia'] = cons1b['periodo_actual'] - cons1b['periodo_anterior']

# Combinar los DataFrames procesados
cons1c = pd.concat([cons1a, cons1b], ignore_index=True)

# Filtrar para un análisis específico
cons1d = cons1c[(cons1c['CONCEPTO'] == "Siniestros liquidados") & (cons1c['Ramo'] == "TRANSPORTE")]

# Agrupar por Ramo y CONCEPTO para obtener resúmenes
cons2 = cons1c.groupby(['Ramo', 'CONCEPTO']).agg({
    'periodo_actual': lambda x: round(x.sum(skipna=True), 2),
    'periodo_anterior': lambda x: round(x.sum(skipna=True), 0)
}).reset_index()
cons2['Variación'] = round(cons2['periodo_actual'] - cons2['periodo_anterior'], 2)
cons2 = cons2.dropna(subset=['Ramo', 'CONCEPTO'])

# Agrupar solo por CONCEPTO para obtener totales
cons3 = cons1c.groupby(['CONCEPTO']).agg({
    'periodo_actual': lambda x: round(x.sum(skipna=True), 2),
    'periodo_anterior': lambda x: round(x.sum(skipna=True), 0)
}).reset_index()
cons3['Variación'] = round(cons3['periodo_actual'] - cons3['periodo_anterior'], 2)
cons3['Ramo'] = 'Total'
cons3 = cons3.dropna(subset=['CONCEPTO'])

# Combinar ambos DataFrames de resumen
cons4 = pd.concat([cons2, cons3], ignore_index=True)

# Escribir datos en Excel (asumiendo que 'wb' ya está definido o creado)
if 'wb' not in locals():
    wb = openpyxl.Workbook()

# Crear una nueva hoja o usar una existente
if "Validación por ramo" in wb.sheetnames:
    sheet = wb["Validación por ramo"]
else:
    sheet = wb.create_sheet("Validación por ramo")

# Escribir los encabezados
for col_num, column_title in enumerate(cons4.columns, 1):
    sheet.cell(row=1, column=col_num, value=column_title)

# Escribir los datos
for row_num, row in enumerate(cons4.values, 2):
    for col_num, cell_value in enumerate(row, 1):
        sheet.cell(row=row_num, column=col_num, value=cell_value)


# EXPORTAR BD ANALISIS 

In [222]:
# Convertir a dataframe y renombrar columnas
bd_analisis_cuentas = pd.DataFrame(bd_analisis_cuentas)
bd_analisis_cuentas.columns = ['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 'Nombre Cuenta',
                              'Saldos', 'Saldos del mes pasado', 'Diferencia mes a mes', 'Diferencia %', 'mes', 'notas']

# Convertir estructura a dataframe y renombrar columnas
estructura = pd.DataFrame(estructura)
estructura.columns = ['Unidad de captura', 'Subcuenta', 'Saldo', 'Nota']

# Agregar hojas al libro de Excel existente (asumiendo que 'wb' ya está definido)
if "Análisis de cuentas" in wb.sheetnames:
    wb.remove(wb["Análisis de cuentas"])
wb.create_sheet("Análisis de cuentas")

if "Estructura" in wb.sheetnames:
    wb.remove(wb["Estructura"])
wb.create_sheet("Estructura")

if "Validación reaseguro1" in wb.sheetnames:
    wb.remove(wb["Validación reaseguro1"])
wb.create_sheet("Validación reaseguro1")

if "Validación coaseguro1" in wb.sheetnames:
    wb.remove(wb["Validación coaseguro1"])
wb.create_sheet("Validación coaseguro1")

# Escribir datos en las hojas
# Análisis de cuentas
sheet = wb["Análisis de cuentas"]
for col_num, column_title in enumerate(bd_analisis_cuentas.columns, 1):
    sheet.cell(row=1, column=col_num, value=column_title)
for row_num, row in enumerate(bd_analisis_cuentas.values, 2):
    for col_num, cell_value in enumerate(row, 1):
        sheet.cell(row=row_num, column=col_num, value=cell_value)

# Estructura
sheet = wb["Estructura"]
for col_num, column_title in enumerate(estructura.columns, 1):
    sheet.cell(row=1, column=col_num, value=column_title)
for row_num, row in enumerate(estructura.values, 2):
    for col_num, cell_value in enumerate(row, 1):
        sheet.cell(row=row_num, column=col_num, value=cell_value)

# Validación reaseguro1
sheet = wb["Validación reaseguro1"]
for col_num, column_title in enumerate(INFORMACION_REASEGURO.columns, 1):
    sheet.cell(row=1, column=col_num, value=column_title)
for row_num, row in enumerate(INFORMACION_REASEGURO.values, 2):
    for col_num, cell_value in enumerate(row, 1):
        sheet.cell(row=row_num, column=col_num, value=cell_value)

# Validación coaseguro1
sheet = wb["Validación coaseguro1"]
for col_num, column_title in enumerate(INFORMACION_COASEGURO.columns, 1):
    sheet.cell(row=1, column=col_num, value=column_title)
for row_num, row in enumerate(INFORMACION_COASEGURO.values, 2):
    for col_num, cell_value in enumerate(row, 1):
        sheet.cell(row=row_num, column=col_num, value=cell_value)

# Guardar el libro de Excel
wb.save("MS_GENERALES.xlsx")